# AimBig Star Ratings Prediction 

### Extracting the Data - for September 2022 Star Rating (1)
- From 1st Star Rating assessment period (for the Sept 2022 Star Rating)
- Assessment period: 1/10/20-7/10/22 (8 quarters)

In [59]:
#Packages:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')


### Analysing from Participant -> Site -> Contract Level 


## Step 1) Actual Performance vs. Expected Performance 
Performance: numerator/denominator for caseload per contract
- Consider DMS and ESS contracts separately
- Only focusing on full outcomes (13, 26, 52 week full outcomes)

- Calculating the performance (actual vs expected) every quarter 

Placement_status: if participant has commenced employment
Placement_Phase: PPS: post-placement-support

Quarterisation:
- Look at example from government pdf (if a person doesn't achieve) - Figure 1
- Check which records is included for a participant 
- Pre-quarterisation: Looking at a participants overall period with a contract (from start date -> end date)
    - This period doesn't consider the start date to end date (broken up by quarters)
- Post-quarterisation: Breaking up the pre-quarterisation period (from start date - to end date) by quarters. Looking at whether a participant has achieved an outcome in each quarter (within the start-end date). If they haven't, then the denominator = 1, and overall denominator += 1. 


### Placement Status Code
- After each quarter, the placement status code is recorded. It is sustained if there are no changes to the status code since the previous quarter. 

Working: 
- Need to be still in assistance at the end of the two year rolling period.
COM_EA: commenced employment with assistance
COM_PPS: commenced employment with post placement support 
COM_OS: commenced employment with ongoing support 
TRN_PPS: transferred with post placement support 
TRN_EA: transferred with employment assistance



- Include these outcomes - given the date period

Exited:
- Exited the site during the two year rolling period
- Include both of these:
EXT_EA: exited the site with assistance, or exited assistance
EXT_PPS: exited the site with post placement support 

exited the Site prior to the two year rolling period but have an associated Outcome which was lodged during the two year period at the Site; or
- Get this from claims 
- Note: this won't be from lifecycle file

exited prior to the two year rolling period but had an ongoing support employment verification date which is within the two year rolling period.
Variables: OSA Date, OSA Due Date (needs to be in the 2 year rolling period)
- don't denominate or numerate

SUS_EA: suspended with employment assistance (Counted in denominator if completed more than 13 weeks with us. Includes suspension period)


### Pathway Meaning
- Include a table with numerator/denominator boolean columns -> initial plan
- Determining whether a combination should be included in a numerator/denominator or both

## Claims and Recording Outcomes
- The COMMENCEMENT_DATE column: marks the date from which a participant is expected to record an outcome. 
- Numerates: 13/26/52 weeks from COMMENCEMENT_DATE - an outcome is achieved.
- COMMENCEMENT_DATE changes based on the PLACEMENT_SEQUENCE_NUMBER.
- Participant needs to numerate a 13wk full outcome before numerating a 26wk full outcome.
- OUTCOME_TYPE: employment only
- Don't count the service fee.
- Edge case: If no claim is made 13wks after the COMMENCEMENT_DATE, but a 26wk claim is made after 13wk - sometimes 13wk claim is made on the same date (late).
- Service fee claims are not counted in numerator
- We get outcome claims when participant is in the PPS phase.
- Once a participant completes 26 weeks - they denominate in 26 weeks, etc.
- Calculating num/denom separately for each full outcome type


##### Loading Data Files, Pre-processing
- Main data file: apv_stream_participants_versions
- Renaming columns, dropping missing commencement dates

In [60]:
## Loading the comb df in:
df_comb =  pd.read_csv('C:/Users/AManalo/OneDrive - Arriba Group/Desktop/Star Ratings - AimBig/apv_stream_participant_versions.csv') # reading from github repo

df_comb = df_comb.rename(columns={'Exit_Date' : 'EXIT_DATE', 'placement_type' : 'PLACEMENT_TYPE'})

# drop rows where COMMENCEMENT_DATE_NEW is NULL -> no valid placements
df_comb = df_comb[df_comb['COMMENCEMENT_DATE'].isna() == False]
# drop None
df_comb = df_comb[df_comb['COMMENCEMENT_DATE'] != None]

In [61]:
df_comb.head(10) # preview of dataframe

,COMMENCEMENT_DATE,Program_Type_Code,jr_jobseeker_id,Job_Seeker_ID,plac_jobseeker_ID,PLACEMENT_TYPE,Program,Contract_ID,Site_Name,ESA_Code,...,CLAIM_AVAILABLE_DATE,Claim_Rate_Type,Claim_Status,Claim_ID,Claim_Type,OUTCOME_TYPE,CLAIM_START_DATE,CLAIM_END_DATE,overridden_outcome_started_on,pps_start_date
0,16/06/2022 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,NaN,NaN,NaN,91581,Des week 52,NaN,NaN,NaN,NaN,16/06/2022
1,16/06/2022 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,15/09/2022 0:00,DES 26wk Full Outcome,Approved,91580,Des week 26,Employment,15/09/2022 0:00,14/12/2022 0:00,NaN,16/06/2022
2,16/06/2022 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,NaN,DES 4wk Outcome,Approved,91578,Des week 4,Employment,13/06/2022 0:00,10/07/2022 0:00,NaN,16/06/2022
3,16/06/2022 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,16/06/2022 0:00,DES 13wk Full Outcome,Approved,91579,Des week 13,Employment,16/06/2022 0:00,14/09/2022 0:00,NaN,16/06/2022
4,22/06/2020 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,21/12/2020 0:00,DES 52wk Full Outcome,Approved,29799,Des week 52,Employment,21/12/2020 0:00,20/06/2021 0:00,NaN,22/06/2020
5,22/06/2020 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,NaN,DES 4wk Outcome,Approved,29796,Des week 4,Employment,22/06/2020 0:00,19/07/2020 0:00,NaN,22/06/2020
10,28/02/2019 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,NaN,DES 52wk Full Outcome,Approved,14777,Des week 52,Employment,26/02/2019 0:00,0001-01-01 00:00:00,NaN,28/02/2019
11,28/02/2019 0:00,DMS,1987,1564000004,1987,Vacancy Placement,Dms4,0212805A,AimBig Employment AITKENVALE,4TOW,...,NaN,DES 4wk Outcome,Approved,14768,Des week 4,Employment,26/02/2019 0:00,25/03/2019 0:00,NaN,28/02/2019
12,21/12/2021 0:00,DMS,10597,6372223709,10597,Vacancy Placement,DMS,0212805A,AimBig Employment AITKENVALE,4TOW,...,22/03/2022 0:00,DES 26wk Full Outcome,Approved,77219,Des week 26,Employment,22/03/2022 0:00,20/06/2022 0:00,NaN,21/12/2021
13,21/12/2021 0:00,DMS,10597,6372223709,10597,Vacancy Placement,DMS,0212805A,AimBig Employment AITKENVALE,4TOW,...,NaN,DES 4wk Outcome,Approved,77217,Des week 4,Employment,22/11/2021 0:00,19/12/2021 0:00,NaN,21/12/2021


The code cell below:
- Take a subset of the columns from df_comb and assigns it to df_pre
- Convert variables to appropriate datatypes
- Replacing out of bounds dates with 1/01/2001 date

In [62]:
# Assigning it to df_pre
df_pre = df_comb[["Job_Seeker_ID", 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE', 'PLACEMENT_TYPE', 'Program',
       'Contract_ID', 'Site_Name', 'ESA_Code',  'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE', 'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE', 'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE', 'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE']]
df_pre = df_pre.sort_values(by=['Job_Seeker_ID', 'CLAIM_START_DATE'])

# converting data types + converting NaT to out of range dates (i.e. 2001)
df_pre['Job_Seeker_ID'] = pd.to_numeric(df_pre['Job_Seeker_ID'], errors='coerce')
df_pre = df_pre[df_pre['Job_Seeker_ID'].isna() != True]
df_pre['Job_Seeker_ID'] = df_pre['Job_Seeker_ID'].astype('int64')


# need to convert column into string first -> then back to date
dict_types = {'SUSPENSION_FROM_DATE' : 'str', 'COMMENCEMENT_DATE' : 'str', 'CLAIM_START_DATE' : 'str', 'CLAIM_AVAILABLE_DATE' : 'str', 'CLAIM_END_DATE' : 'str', 'SUSPENSION_ACTUAL_END_DATE' : 'str' }
df_pre = df_pre.astype(dict_types)


# fill NA values with out of bounds date
fillVal = pd.to_datetime('1/01/2001', format='%d/%m/%Y')


df_pre['SUSPENSION_FROM_DATE'] = pd.to_datetime(df_pre['SUSPENSION_FROM_DATE'], format='%d/%m/%Y', exact=False)
df_pre['SUSPENSION_FROM_DATE'] = df_pre['SUSPENSION_FROM_DATE'].fillna(fillVal)


df_pre['SUSPENSION_ACTUAL_END_DATE'] = pd.to_datetime(df_pre['SUSPENSION_ACTUAL_END_DATE'], format='%d/%m/%Y', exact=False)
df_pre['SUSPENSION_ACTUAL_END_DATE'] = df_pre['SUSPENSION_ACTUAL_END_DATE'].fillna(fillVal)

df_pre['COMMENCEMENT_DATE'] = pd.to_datetime(df_pre['COMMENCEMENT_DATE'], format='%d/%m/%Y', exact=False)
df_pre['COMMENCEMENT_DATE'] = df_pre['COMMENCEMENT_DATE'].fillna(fillVal)


df_pre['CLAIM_START_DATE'] = pd.to_datetime(df_pre['CLAIM_START_DATE'], format='%d/%m/%Y', exact=False)
df_pre['CLAIM_START_DATE'] = df_pre['CLAIM_START_DATE'].fillna(fillVal)

df_pre['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_pre['CLAIM_AVAILABLE_DATE'], format='%d/%m/%Y', exact=False)
df_pre['CLAIM_AVAILABLE_DATE'] = df_pre['CLAIM_AVAILABLE_DATE'].fillna(fillVal)

df_pre['CLAIM_END_DATE'] = pd.to_datetime(df_pre['CLAIM_END_DATE'], format='%d/%m/%Y', exact=False, errors='coerce')
df_pre['CLAIM_END_DATE'] = df_pre['CLAIM_END_DATE'].fillna(fillVal)




Date Period = 1/10/20-7/10/22
- Replace missing values with 'na', standardise
- Remove duplicate rows from the dataframe
- Create a new variable, 'SUSPENSION_DAYS' that measures the difference between suspension start date and end date

In [63]:
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')


df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].replace(np.nan, 'na', regex=True)
df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].replace('', 'na', regex=True)
df_pre["Claim_Rate_Type"] = df_pre["Claim_Rate_Type"].astype('str')


# remove duplicates - make copy of duplicate free dataframe 
df_pre = df_pre.loc[:,~df_pre.columns.duplicated()].copy()


# getting the period between suspension_from_date and suspension_actual_end_date in weeks
df_pre['SUSPENSION_DAYS'] = (df_pre['SUSPENSION_ACTUAL_END_DATE'] - df_pre['SUSPENSION_FROM_DATE']).apply(lambda x: x/np.timedelta64(1, 'D'))

df_pre['SUSPENSION_DAYS'] = df_pre['SUSPENSION_DAYS'].astype('int64')




### Filling in the 'specialist_site_type_code' blanks - THIS HAS BEEN DONE IN EXCEL MANUALLY
- Filled in using the existing codes with corresponding contract_id and site_description
- First groupBy the object
- Then iterate and fillNa values 

Need to check if this has been done recently -> otherwise can automate this process

## Performance - Numerator
- Aggregate numerator per job_seeker_id
- aggregate numerator for each type of full outcome
- Decide what to do for denominators..

## Denominators
- Duration at assistance is at least 13/26/52 weeks (excluding suspension periods)
- Excludes participants who have achieved a pathway outcome

In [64]:
df_pre['REFERRAL_PHASE_CODE'].drop_duplicates()

426938     EA
427956    PPS
374176     OS
174961     NE
197050    NaN
Name: REFERRAL_PHASE_CODE, dtype: object

- Placement codes categorised into transition, exit, suspension and commence type

In [65]:
# for denominator: include working codes 
# since referral phase code can only be PPS or EA
ref_pps = (df_pre["REFERRAL_PHASE_CODE"] == 'PPS')
ref_ea = (df_pre["REFERRAL_PHASE_CODE"] == 'EA')
ref_os = (df_pre["REFERRAL_PHASE_CODE"] == 'OS')
com = (df_pre["PLACEMENT_STATUS_CODE"] == 'Commenced')
trn = (df_pre["PLACEMENT_STATUS_CODE"] == 'Transferred')

com_ea = com & ref_ea
com_pps = com & ref_pps
com_os = com & ref_os 
trn_pps = trn & ref_pps
trn_ea = trn & ref_ea 


# include exit codes - but check date (exited during 2 year rolling period): 
# just do start of 2018
date_range = df_pre['COMMENCEMENT_DATE'].loc[(df_pre['COMMENCEMENT_DATE'] >= '2020-10-01') & (df_pre['COMMENCEMENT_DATE'] <= '2022-10-07')]


ext = (df_pre["REFERRAL_PHASE_CODE"] == 'Exited')

is_sus = (df_pre['SUSPENSION_DAYS'].isna() == False) # suspension dates exist and check commencement period 


Types of Claims

In [66]:
df_pre['Claim_Rate_Type'].value_counts()

na                                                    269846
DES 4wk Outcome                                        57142
DES 13wk Full Outcome                                  29816
DES 26wk Full Outcome                                  20664
DES 52wk Full Outcome                                   7466
DES 26wk Full Outcome Education                         2309
DES 13wk Full Outcome Education                         2056
DES 13wk Pathway Outcome                                2050
DES 26wk Pathway Outcome                                 586
DES 13wk Pathway Outcome Education                       286
DES 52wk Pathway Outcome                                 268
DES 4wk Outcome Moderate Intellectual Disability P        35
DES 13wk Outcome Moderate Intellectual Disability         28
DES 26wk Outcome Moderate Intellectual Disability         16
DES 13wk Full Outcome Zero Dollar R4                       8
DES 52wk Outcome Moderate Intellectual Disability          6
Name: Claim_Rate_Type, d

Creates a new column, PLACE_REF that combines the placement status code aand referral phase code

In [67]:
# combine the placement_status_code and referral_phase_code :
df_pre['PLACE_REF'] = list(zip(df_pre.PLACEMENT_STATUS_CODE, df_pre.REFERRAL_PHASE_CODE))


### Filtering NaT for - Claim_available_date

- Validating the date period

In [68]:
df_pre['CLAIM_AVAILABLE_DATE'] = pd.to_datetime(df_pre['CLAIM_AVAILABLE_DATE'])
df_pre['COMMENCEMENT_DATE'].min()

Timestamp('2018-05-04 00:00:00')

In [69]:
df_pre['CLAIM_AVAILABLE_DATE'].max()

Timestamp('2022-10-07 00:00:00')

## 13 Week Outcomes (Numerator + Denominator)
+ Creating a dictionary (key: Job_Seeker_ID, values: [all attributes for one row], [all attributes for second row], etc.) 
    - The values in the dictionary are the rows which correspond to the same Job_Seeker_ID -> but stored like so: 
    E.g. for job_seeker_id 1001:
    1001     :  [row 1]
                [row 2]
                [row 3]
    This allows for efficient access when identifying the smallest placement_seq_no for instance -> accessing row 1, rather than iterating through the dataframe. 
    - Can check index of the rows -> need to check whether if row 3 has a 26 week outcome, check if 13 week outcome is in row 2 or row 1. 
    - Check 13 week outcome by adding 13 weeks to commencement date by adding 13x7 days.

In [71]:
df_pre = df_pre.sort_values(by=['COMMENCEMENT_DATE'])
df_pre.head(5)

,Job_Seeker_ID,REFERRAL_PHASE_CODE,PLACEMENT_STATUS_CODE,PLACEMENT_TYPE,Program,Contract_ID,Site_Name,ESA_Code,specialist_site_type_code,COMMENCEMENT_DATE,...,Claim_Rate_Type,Claim_Status,CLAIM_START_DATE,CLAIM_AVAILABLE_DATE,CLAIM_END_DATE,SUSPENSION_FROM_DATE,SUSPENSION_ACTUAL_END_DATE,OUTCOME_TYPE,SUSPENSION_DAYS,PLACE_REF
457888,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
459936,9386301003,NE,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, NE)"
459935,9386301003,NE,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, NE)"
457641,9386301003,PPS,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, PPS)"
459934,9386301003,NE,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, NE)"
459232,9386301003,EA,Commenced,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Commenced, EA)"
459170,9386301003,EA,Suspended,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Suspended, EA)"
459169,9386301003,EA,Suspended,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Suspended, EA)"
459168,9386301003,EA,Suspended,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Suspended, EA)"
458461,9386301003,EA,Exited,Vacancy Placement,DMS,0212812K,AimBig Employment WOY WOY,4CEC,AALL,2018-05-04,...,na,NaN,2001-01-01,2001-01-01,2001-01-01,2020-10-13,2020-11-09,NaN,27,"(Exited, EA)"


Create a groupby object, each group is indexed by the Job_Seeker_ID
- Sort on commencement_date and claim_start_date

In [72]:
df_pre = df_pre.drop_duplicates()
df_pre = df_pre.sort_values(['COMMENCEMENT_DATE', 'CLAIM_START_DATE'])

# Job_Seeker_ID is set as the index -> access by "g.index[0]"

g = df_pre.groupby(["Job_Seeker_ID"]) # remove commencement_date with NaT -> check if NaT values, filter these out and remove




- Groups -> one group containing entries that correspond to the same Job_Seeker_ID
srce: https://stackoverflow.com/questions/22691010/how-to-print-a-groupby-object


## 13 Week Numerator and Denominator Function - Pre-Quarterisation
- Get the lowest index value of the group (for each list)
- Create a new dataframe df_13: [Job_Seeker_ID, Num_13, Den_13] 
    - Contains the 13 week numerator and denominator for corresponding job seeker id
- Use pd.DateOffset instead of timedelta(days=....)
- Excludes in the denominator: participants who have achieved a 13 week pathway outcome

In [73]:
# list that contains the numerator codes: 
com_ea = com & ref_ea
com_pps = com & ref_pps
com_os = com & ref_os 
trn_pps = trn & ref_pps
trn_ea = trn & ref_ea 
num_codes = [('Commenced', 'EA'), ('Commenced', 'PPS'), ('Commenced', 'OS'), ('Transferred', 'PPS'), ('Transferred', 'EA')]


#EXT_EA: exited the site with assistance, or exited assistance
#EXT_PPS: exited the site with post placement support 
den_codes = [('Exited', 'EA'), ('Exited', 'PPS'), ('Transferred', 'PPS'), ('Transferred', 'EA')]

# participant has commenced at a site:
com_codes = [('Commenced', 'EA'), ('Commenced', 'PPS')]


In [74]:
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

In [75]:
df_pre['PLACEMENT_STATUS_CODE'].value_counts()

Commenced      58835
Pending        27061
Exited         22238
Suspended      13562
Transferred     4631
In Progress      539
Enquiry            6
Name: PLACEMENT_STATUS_CODE, dtype: int64

In [76]:
def get_13_outcome(jobSeekerEntry, num_codes, den_codes, com_codes):
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry
    id = group.index.values.astype(int)[0] 
    # initialise numerator and denominator 
    num_13 = 0
    den_13 = 0
    df_13 = pd.DataFrame()
    # place id into dataframe
    df_13['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    
    endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
    startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
    
    # TEMP ----------- REMOVE
    df_13['Site_Name'] = [group.Site_Name.values[0]]
    df_13['Contract_ID'] = [group.Contract_ID.values[0]]
    df_13['specialist_site_type_code'] = [group.specialist_site_type_code.values[0]]

    # TEMP -------------------
    week_13 = pd.DateOffset(days = 91)
    # Check number of entries in group
    targetStr = 'DES 13wk Full Outcome'
    num_entries = int(len(jobSeekerEntry))

    # CLAIM TYPES - 
    eduStr = ['DES 13wk Full Outcome Education', 'DES 13wk Pathway Outcome Education'] # denominate
    noClaim = 'na'
    pathwayStr = 'DES 13wk Pathway Outcome' # doesn't denominate, doesn't numerate
    valid_claims = ['Approved', 'Pending', 'Lodged'] #but excluding lodged, pending or approved FOR EDUCATION 13 week full outcomes
    exitDate = group.EXIT_DATE.values[0] # if exit date exists -> 

    # now loop ->
    j = 0
    comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0])
    exitDate = pd.to_datetime(group.EXIT_DATE.values[0])



    ### check that COMMENCEMENT_DATE IS NOT '2001-01-01'
    inavlid = pd.to_datetime('2001-01-01', format='%Y-%m-%d')
    if comDate == inavlid:
        df_13['Num_13'] = 0
        df_13['Den_13'] = 0
        return df_13
    
    if exitDate < startBounds and comDate < exitDate: # participant exits before start of assessment period, and does not commence after exiting
        df_13['Num_13'] = 0
        df_13['Den_13'] = 0
        return df_13
         
         
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_13 = comDate + week_13
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_13): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_13 = comDate + week_13 + sus_days
            else:
                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] == targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                
                # check if participant exits before claim_available_date ->
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_13):
                    
                    num_13 = 0 
                    den_13 = 0
                    
                else:
                    num_13 += 1
                    den_13 = 1
                    df_13['Num_13'] = [num_13]
                    df_13['Den_13'] = den_13
                    return df_13 

            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_13 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_13: # duration in assistance is less than 13 weeks
                    num_13 = 0 
                    den_13 = 0
                   
                else: # check duration in assistance is at least period_13 
                    num_13 = 0
                    den_13 = 1 #education outcomes are counted in the denominator
            elif group.Claim_Rate_Type.values[j] == pathwayStr and (group.Claim_Status.values[j] in valid_claims): # denominator excludes pathway outcomes
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_13):
                    num_13 = 0 
                    den_13 = 0
                else:
                    num_13 += 0 
                    
                    
                    break
            elif group.Claim_Rate_Type.values[j] == noClaim: 
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            
                if (exitDate < period_13):
                    num_13 = 0 
                    
                else:
                   
                    num_13 += 0 
                    den_13 = 1
                   
            else: # either 26, 52 week claim
                num_13 = 0
                
            
        
        # has not commenced ->
        else: 
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_13 = comDate + week_13
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_13): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_13 = comDate + week_13 + sus_days

            # check if code is SUS_EA: (Counted in denominator if completed more than 13 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_13:
                    if (group.Claim_Rate_Type.values[j] == targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_13 <= startBounds) or (period_13 >= endBounds):
                                num_13 = 0 
                                den_13 = 0 # out of bounds
                                
                        else:
                            # then doesn't denominate: 
                            num_13 = 1
                            den_13 = 1
                            

                      
                       
                    else:
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_13 <= startBounds) or (period_13 >= endBounds):
                            num_13 = 0 
                       
                            
    
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 13wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_13 = 0 
                  
                       
                    
                else: # no claim available date:
                    if group.Claim_Rate_Type.values[j] == 'na':
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_13:
                            num_13 = 0
                            den_13 = 1
                            break

                    else:
                        num_13 = 0
                        

            else: # TRN or EXT
                
                ext_tfn = ['Exited', 'Transferred']
              
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                
            

                
                # check claim rate types ->
                if (group.Claim_Rate_Type.values[j] == targetStr) and (group.Claim_Status.values[j] in valid_claims): # valid claim
                
                    if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                        if comDate > exitDate: # if participant commences again after exiting then can record outcome ->
                            num_13 += 1
                            den_13 = 1
                            df_13['Num_13'] = [num_13]
                            df_13['Den_13'] = den_13
                            return df_13  
                        else:
                        
                            num_13 = 0
                           
                    else:
                        num_13 += 1
                        den_13 = 1
                        df_13['Num_13'] = [num_13]
                        df_13['Den_13'] = den_13
                        return df_13  

                elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_13 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                    if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                        if comDate > exitDate: # if participant commences again after exiting then can record outcome ->
                            den_13 = 1
                            df_13['Num_13'] = [num_13]
                            df_13['Den_13'] = den_13
                        else:
                            num_13 = 0
                            

                    else:
                        num_13 = 0
                        den_13 = 1


                elif group.Claim_Rate_Type.values[j] == pathwayStr and (group.Claim_Status.values[j] in valid_claims): # denominator excludes pathway outcomes
              
                    den_13 = 0
                    df_13['Num_13'] = [num_13]
                    df_13['Den_13'] = den_13
                    break
                       
                 

                elif group.Claim_Rate_Type.values[j] == noClaim: 
                    if exitDate < group.CLAIM_AVAILABLE_DATE.values[j]:
                        num_13 = 0
                       
                    else:
                        
                        num_13 += 0 
                        den_13 += 1

                else: # either 26, 52 week claim
                    
                    num_13 = 0
                   
                       
                
        
        j += 1

    if den_13 > 1:
        den_13 = 1
        
    df_13['Num_13'] = [num_13]
    df_13['Den_13'] = den_13
    return df_13
 






Testing a job seeker passed to the pre-quarter 13 week function:

In [77]:
### Continuous 
jobEntry = g.get_group(9962285509)
get_13_outcome(jobEntry, num_codes, den_codes, com_codes)

,Job_Seeker_ID,Site_Name,Contract_ID,specialist_site_type_code,Num_13,Den_13
0,9962285509,AimBig Employment MOONEE PONDS,0212817E,AALL,0,0


In [78]:
dups_color = df_pre.pivot_table(columns=['Job_Seeker_ID'], aggfunc='size')
print (dups_color)
# need to drop duplicates such that Job_Seeker_ID is unique: 


Job_Seeker_ID
44710         20
61780         12
70560         20
82960         32
85140         48
              ..
9983739909    16
9985676409    48
9986691809    36
9996900006    80
9999230109    10
Length: 4970, dtype: int64


### Aggregate 13_week_outcome function

Looping through df_comb and passing each group to the get_13_outcome function

In [79]:
# Number of distinct Job_Seeker_IDs from df_pre:
distinct = df_pre['Job_Seeker_ID'].unique()


In [80]:
# adds outcome to df_13 ->

def agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes):
    df_13 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_13', 'Den_13', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome

    ids = df_pre['Job_Seeker_ID'].unique()
    for id in ids:
        if math.isnan(id) or id == 0:
            pass
        else:
            df_entry = g.get_group(id)
            f = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
            df_13 = df_13.append(f)
            

    return df_13 # returns output from get_13_outcome 

# map outcomes to dataframe -> one pass


x = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)

In [81]:
x # output dataframe from aggregation function 

,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,9386301003,0,1,AimBig Employment WOY WOY,0212812K,AALL
0,8653691003,0,1,AimBig Employment FAIRFIELD,0212899A,AALL
0,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL
0,6498977509,0,0,AimBig Employment LEICHHARDT,0212845J,AALL
0,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL
...,...,...,...,...,...,...
0,2836891309,1,1,AimBig Employment CORIO,0212862K,AALL
0,8771045004,0,1,AimBig Employment FRANKSTON,0212818F,MENH
0,15325790,1,1,AimBig Employment MACKAY,0212803K,MUSK
0,690092007,1,1,AimBig Employment Pty Ltd SUNBURY,0212817E,AALL


In [82]:
x = x.drop(columns=['Job_Seeker_ID'])
x = x.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13', 'Den_13'].sum()


In [83]:
x # dataframe grouped by contract

Num_13  \
Site_Name                          Contract_ID specialist_site_type_code           
AimBig Employment AITKENVALE       0212804K    MENH                           16   
                                   0212805A    MUSK                            9   
AimBig Employment AUBURN           0212841E    AALL                           28   
                                   0212898K    AALL                            9   
AimBig Employment BALLARAT CENTRAL 0212859D    MENH                            3   
...                                                                          ...   
AimBig Employment WYONG            0212881B    AALL                            5   
Aimbig Employment LAUNCESTON       0212865B    (blank)                         6   
                                   0212893F    AALL                            5   
Aimbig Employment RICHMOND         0212850F    AALL                            3   
Aimbig Employment WINDSOR          0212903C    (blank)                         1   

                                                                          Den_13  
Site_Name                          Contract_ID specialist_site_type_code          
AimBig Employment AITKENVALE       0212804K    MENH                           32  
                                   0212805A    MUSK                           10  
AimBig Employment AUBURN           0212841E    AALL                           45  
                                   0212898K    AALL                           23  
AimBig Employment BALLARAT CENTRAL 0212859D    MENH                           10  
...                                                                          ...  
AimBig Employment WYONG            0212881B    AALL                           17  
Aimbig Employment LAUNCESTON       0212865B    (blank)                        12  
                                   0212893F    AALL                           11  
Aimbig Employment RICHMOND         0212850F    AALL                            4  
Aimbig Employment WINDSOR          0212903C    (blank)                         1  

[252 rows x 2 columns]

### Testing sites -> job seekers - for a particular Contract
- jobseeker_test.csv = contains jobseeker ids of participants under a particular contract
- Ids have been manually extracted from apv_stream_participants_versions after filtering for a contract
- Read csv in, and pass as a dataframe to test the 13 week outcomes function

In [128]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')

x = agg_outcomes_13(jobSeekers, g, num_codes, den_codes, com_codes)

x['Num_13'].sum()
x

,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,7006568909,0,1,AimBig Employment WYONG,0212881B,AALL
0,306053509,0,1,AimBig Employment WYONG,0212881B,AALL
0,9397837003,1,1,AimBig Employment WYONG,0212881B,AALL
0,8126968509,1,1,AimBig Employment WYONG,0212881B,AALL
0,7922400909,0,1,AimBig Employment WYONG,0212881B,AALL
0,1708521004,0,1,AimBig Employment WYONG,0212881B,AALL
0,759600003,1,1,AimBig Employment WYONG,0212881B,AALL
0,799511609,0,1,AimBig Employment WYONG,0212881B,AALL
0,6170328003,0,1,AimBig Employment WYONG,0212881B,AALL
0,3777340909,1,1,AimBig Employment WYONG,0212881B,AALL


In [85]:
x['Den_13'].sum()

36

### Pre-Quarterisation Output File:
Running the pre-quarter 13 week function and creating a csv file passed to the repository. 

In [86]:
df_13 = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)

print('Number of IDs for 13 week outcomes: ' + str(len(df_13)))
df_13

Number of IDs for 13 week outcomes: 4970


,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code
0,9386301003,0,1,AimBig Employment WOY WOY,0212812K,AALL
0,8653691003,0,1,AimBig Employment FAIRFIELD,0212899A,AALL
0,7870290004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL
0,6498977509,0,0,AimBig Employment LEICHHARDT,0212845J,AALL
0,6638331004,0,0,AimBig Employment NORTH LAKES,0212795D,AALL
...,...,...,...,...,...,...
0,2836891309,1,1,AimBig Employment CORIO,0212862K,AALL
0,8771045004,0,1,AimBig Employment FRANKSTON,0212818F,MENH
0,15325790,1,1,AimBig Employment MACKAY,0212803K,MUSK
0,690092007,1,1,AimBig Employment Pty Ltd SUNBURY,0212817E,AALL


In [87]:
# output file
df_13.to_csv('df_13_pre.csv')


### Post-Quarterisation for 13 Week Outcomes 
- Numerator doesn't change, only the DENOMINATOR
- Check - The numerator achieved date -> 13 weeks after commencement date 
- Denominate each quarter if they do not numerate. For each quarter a participant is eligible to record an outcome, i.e. been in assistance at least 13 weeks - if no outcome is recorded, then the participant continues denominating. 
- If the quarter falls outside the 2 year period - record is not included
- not included if time to achieve an outcome < 13 weeks
- Need to break up the 2 year rolling period into quarters -> 8 quarters (Based on business year)


**Given the 2 year rolling period - breaking the period up to quarters**

- Format: nested list [[start_date, end_date],...]
- 8 quarters
- Index match between q_start_dates and q_end_dates, e.g. 1st quarter is [q_start_dates[0], q_end_dates[0]]

In [88]:
q_start_dates = pd.date_range('2020-10-01', '2022-10-07', freq='BQS')
q_end_dates = pd.date_range('2020-10-01', '2022-10-07', freq='BQ-DEC')
print("All the business quarterly begin dates of 2020-22:")
print(q_start_dates.values)
print("\nAll the business quarterly end dates of 2020-22:")
print(q_end_dates.values)

print("\n")

All the business quarterly begin dates of 2020-22:
['2020-10-01T00:00:00.000000000' '2021-01-01T00:00:00.000000000'
 '2021-04-01T00:00:00.000000000' '2021-07-01T00:00:00.000000000'
 '2021-10-01T00:00:00.000000000' '2022-01-03T00:00:00.000000000'
 '2022-04-01T00:00:00.000000000' '2022-07-01T00:00:00.000000000'
 '2022-10-03T00:00:00.000000000']

All the business quarterly end dates of 2020-22:
['2020-12-31T00:00:00.000000000' '2021-03-31T00:00:00.000000000'
 '2021-06-30T00:00:00.000000000' '2021-09-30T00:00:00.000000000'
 '2021-12-31T00:00:00.000000000' '2022-03-31T00:00:00.000000000'
 '2022-06-30T00:00:00.000000000' '2022-09-30T00:00:00.000000000']




Manually extracted quarter dates from the two lists above:

Quarters:
Q1: '2020-10-01', '2020-12-31'
Q2: '2021-01-01', '2021-03-31'
Q3: '2021-04-01', '2021-06-30'
Q4: '2021-07-01', '2021-09-30'
Q5: '2021-10-01', '2021-12-31'
Q6: '2022-01-03', '2022-03-31'
Q7: '2022-04-01', '2022-06-30'
Q8: '2022-07-01', '2022-09-30'

Convert each quarter into a datetime

In [89]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]
z = 0
s = 0
e = 1
while z < len(qtrs):
    start = datetime.strptime(qtrs[z][s], '%Y-%m-%d')
    qtrs[z][s] = start
    end = datetime.strptime(qtrs[z][e], '%Y-%m-%d')
    qtrs[z][e] = end
    
    z += 1

### Edge Cases for Post Quarterisation:
- If participant does not record a 13 week outcome ever -> denominates the max 8 times.
- If time to achieve an outcome is less than 13 weeks -> don't denominate. -> but this should be in the df_13 already 
- If quarter falls outside of the two year assessment period, then not included.


For 13 week pathway outcomes:
- If this type of outcome is recorded, it is not included in the denominator -> ignored
- However if a participant is at the site for at least 13 weeks and no FULL outcome is recorded -> continues denominating until full outcome is recorded 

### 13 week - Post Quarterisation Function

In [146]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]


endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')

startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

def post_quarter_13(df_13, df_pre, g, qtrs):
    '''Post quarterisation performance for 13 week full outcomes. Receives df_pre as an input, and outputs the post_quarterised denominators for each eligible participant.
    df_13 is used to check if a participant has numerated, df_pre is used to feed in the job_seeker_id to 'g' - the groupBy object and check whether the job seeker denominates - if no numerator in the df_13 exists.'''
    # maximum amount of times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_13', 'Den_13', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # Dataframe template for result
    
    claimExists = False
    
    i = 0
    while i < len(df_13):
       
        js_id = df_13.Job_Seeker_ID.values[i]
        siteName = df_13.Site_Name.values[i]
        contractID = df_13.Contract_ID.values[i]
        specialistCode = df_13.specialist_site_type_code.values[i]
        num_13 = df_13.Num_13.values[i] 
        den_13 = df_13.Den_13.values[i]
        Den_13 = 0
    
        Num_13 = num_13
            
        
        x = 0
        group = g.get_group(js_id)
        n = len(group)
        week_13 = pd.DateOffset(days = 91)
        stopLoop = False
        while x < n or not stopLoop:
            comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0], format='%Y/%m/%d %H:%M') # commencement date should be the first index 
            claim = group.Claim_Rate_Type.values[x]
            availDate = pd.to_datetime(group.CLAIM_AVAILABLE_DATE.values[x])
            exitDate = pd.to_datetime(group.EXIT_DATE.values[x], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
            invalid_date = pd.to_datetime('2001-01-01', format='%Y-%m-%d')
            sus_start = pd.to_datetime(group.SUSPENSION_FROM_DATE.values[x])
            sus_end = pd.to_datetime(group.SUSPENSION_ACTUAL_END_DATE.values[x])
            sus_days = group.SUSPENSION_DAYS.values[x].dt.days
          
            if den_13 == maxDen:
                pass # return -> denominated maxDen = 8 times, or maxNum/maxDen
            
            if num_13 == 0 and den_13 == 0:
                Num_13 = 0
                Den_13 = 0
                stopLoop = True
                break 
            
        
            elif comDate <= invalid_date:
                Num_13 = 0
                Den_13 += 0
            else: 
                # period_13 is initialised in this block
                # looping through the outcomes
                if (group.PLACEMENT_STATUS_CODE.values[x] == 'Exited') and (group.SUSPENSION_DAYS.values[x] > 0): # current entry is showing Exited -> check if exitDate is  
                    #suspension handling: if the suspension period occurs in one of the current quarters
                    period_13 = comDate + week_13 
               

                # loop through the quarters and compare with claim_available_date -> check which quarter it occurs in
                else:
                    
                    if (claim == 'DES 13wk Full Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 13 week outcome claim exists - validate the number for the denominator -> reset                    
                        Num_13 = 1
                        pass
                        
                    elif claim == 'DES 52wk Full Outcome' or claim == 'DES 26wk Full Outcome':
                        Num_13 += 0 
                        Den_13 += 0 
                        pass
                    # 13 week outcome may already be recorded -> or may be in later entries
                    
                    else:
                        period_13 = comDate + week_13
                    # check if a 13 week full outcome exists in ANY of the group values
                        
                    z = 0
                    s = 0
                    e = 1
                    while z < len(qtrs) and not stopLoop:
                        start = pd.to_datetime(qtrs[z][s])
                        end = pd.to_datetime(qtrs[z][1])
                        
                        if z == len(qtrs) - 1:  # This is the last iteration
                            stopLoop = True
                        if Den_13 == maxDen:
                            stopLoop = True
                            break
                        else:
                            
                            
                            if 'DES 13wk Full Outcome' in group.Claim_Rate_Type.values:
                                # if the iteration -> l
                                # +oop is at the beginning of the current group -> get the commencement date of the FIRST row for the 13 week full outcome: 
                                outcomeInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[outcomeInd]
                                # get EXIT date for this here
                        
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[outcomeInd], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
                                ## TO-DO - investigate whether a SUSPENSION period occurs here and if exitDate <= period_13 (comDate + suspension_period + 13 weeks)
                            elif ('DES 13wk Full Outcome' not in group.Claim_Rate_Type.values ) and ('DES 13wk Pathway Outcome' in group.Claim_Rate_Type.values):    # all values are 13 week PATHWAY outcomes
                                pathwayInd = (group.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[pathwayInd]
                    
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[pathwayInd], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
                            # check here how many times the participant denominates ->
                            if (comDate >= start) and (comDate <= end) and (group.PLACEMENT_STATUS_CODE.values[x] == 'Commenced'): # check if the commencement date is in the current quarter: 
                                # check if suspension occurs in the current quarter ->
                                if (sus_days > 0): # check suspension cases:
                                    # 1) If suspension starts and ends in the current quarter
                                    if sus_start >= start and sus_end <= end:
                                        period_13 += sus_days # add the suspension days
                                    # 2) If suspension starts in a previous quarter and ends in the current, however suspension would occur previous to commencement. Only add days that are after commencement
                                    elif sus_start <= start and (start <= sus_end <= end):
                                        sus_days = sus_end - comDate # only take from comDate 
                                        # Update period_13:
                                        period_13 += sus_days
                                    # 3) If suspension starts in current quarter and ends in future quarter
                                    elif start <= sus_start <= end and sus_end >= end:
                                        # update sus_days to just include  
                                        
                            
                                if claim == 'DES 13wk Pathway Outcome':
                                    Den_13 = 0
                            
                                elif (exitDate >= start) and (exitDate <= end) and (period_13 >= start) and (period_13 <= end): #exit date is in the current quarter -> then denominates and then exists
                                
                                    Den_13 += 1 
                                    stopLoop = True
                                    break
                                elif (availDate >= start) and (availDate <= end) and (period_13 >= start) and (period_13 <= end): # in the current quarter -> exit date not counted
                                    Den_13 += 1
                                   
                                #add here if exit before period_13
                                elif (exitDate < period_13):
                                
                                    Den_13 = 0
                                    stopLoop = True
                                    break
                                else:
                                    Den_13 += 1
                                    


                            else: # commencement date has already passed, OR commencement date is later
                                # need to stop at participant EXIT_DATE - get exit date of the 13 week full outcome or pathway outcome:
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[0], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
                                period_13 =  comDate + week_13
                                
                                if (comDate > end): # commencement date is later ->
                                    pass
                                    
                                elif exitDate < period_13:
                                    Den_13 = 0
                                    stopLoop = True
                                    break 
                                
                                else: 
                                    
                                    # check if exitDate is within the current quarter being traversed through
                                    if (exitDate >= start) and (exitDate <= end):
                                        if period_13 >= start and period_13 <= end: # duration at least 13 weeks OCCURS during this quarter and they exit in this quarter
                                            Den_13 += 1
                                            stopLoop = True
                                            break
                                        elif exitDate < period_13:
                                            Den_13 = 0
                                            stopLoop = True
                                            break
                                        else:
                                
                                            Den_13 += 1
                                            stopLoop = True
                                            break
                                            
                                    elif (period_13 >= start and period_13 <= end) or (period_13 < start) and not (start <= exitDate <= end):
                                        
                                        if (availDate >= start and availDate <= end) and claim == 'DES 13wk Full Outcome': # claim is in current quarter
                                            Den_13 += 1
                                            stopLoop = True
                                            break
                                        else:
                                            
                                            Den_13 += 1
                                            
                                       
                          
                                
                        z += 1
                        
                   
                        
                    
                

                  

                x += 1
        

        if Den_13 >= maxDen:
            Den_13 = 8  # set threshold
        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_13_full' : num_13, 'Den_13_full' : Den_13, 'Site_Name' : [siteName], 'Contract_ID' : [contractID], 'specialist_site_type_code' : [specialistCode]})
        df_post = df_post.append(data)

            

            
        
        i += 1
            





    return df_post # returns post-quarterisation dataframe 




SyntaxError: invalid syntax (3155109160.py, line 120)

**Testing the post quarterisation function**
- Using jobseeker_test.csv, contains job seeker ids from a particular contract

In [143]:
# need to filter df_13 that is passed to the function
# this job seekers exits before period_13

jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')
jobSeekers = jobSeekers[jobSeekers['Job_Seeker_ID'] == 1237048409]
df_test = agg_outcomes_13(jobSeekers, g, num_codes, den_codes, com_codes)

# calling post_quarter function
post_13_test = post_quarter_13(df_test, df_pre, g, qtrs)



nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere
nhere


In [140]:
post_13_test['Den_13_full'] # AITEN A = 24

0    7.0
Name: Den_13_full, dtype: float64

In [141]:
post_13_test

,Job_Seeker_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_13_full,Den_13_full
0,1237048409,NaN,NaN,AimBig Employment WYONG,0212881B,AALL,0.0,7.0


In [131]:
post_13_test = post_13_test.groupby(['Job_Seeker_ID', 'Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13', 'Den_13'].sum()
post_13_test = post_13_test.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13', 'Den_13'].sum()
post_13_test

,,,Num_13,Den_13
Site_Name,Contract_ID,specialist_site_type_code,,
AimBig Employment WYONG,0212881B,AALL,0,0


**Creating the post_quarter_13 output file**
- Placed in repository for downstream tasks

In [94]:

x = agg_outcomes_13(df_pre, g, num_codes, den_codes, com_codes)
df_13_post = post_quarter_13(x, df_pre, g, qtrs)

df_13_post.to_csv('C:/Users/AManalo/star_ratings_new/df_13_post.csv')
df_13_post = df_13_post.groupby(['Job_Seeker_ID', 'Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full'].sum()
df_13_post = df_13_post.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full'].sum()

df_13_post.to_csv('C:/Users/AManalo/star_ratings_new/13_week_outcomes_sites.csv')


# 26 Week Outcomes

### Eligible 26 week outcomes:
Included:
- Full 26 week outcomes (excluding Education - only vacancy )
- Pathway 26 week outcomes (worth 5%)
- Bonus/work assist outcomes 

Excluded:
- Zero dollar outcomes 
- Education outcomes

1) Renaming Num_26 and Den_26 => Num_path_26, Den_path_26, Num_wrkast_26, den_wrkast_26 (work assist/bonus), Num_26 -> Num_full_26, Den_full_26


### 1) Pre-Quarterisation

Difference to the existing pre-quarterisation function: 

    1.  Get the date from the [13 week outcome] claim_available_date - in which the 13 week outcome was achieved (not commencement_date like with 13 week outcome)
        done

    NOW: assuming that the data is correct - continue the pre-quarterisation function 


    2. Add 13 weeks to the date from (1) excluding suspension periods. Check if EXIT_DATE occurs before 26 week is recorded -> excluded. 

    3. If there exists a 26 week eligible outcome within the 2 year rolling period (after the 13_week_claim_available_date) –> NUM_26 = 1, DEN_26 = 1

    4. Otherwise: NUM_26 = 0, DEN_26 = 1

    5. If 26 week outcome ELIGIBILITY DATE (26 weeks from 13 week outcome) falls outside the assessment period, then -> NUM_26 = 0, DEN_26 = 0 

Confirm types of eligible 26 week outcomes to be taken <- from Chiara’s email. 





--------------

### Pre-quarterisation function (26 week outcome)
Same as the pre-quarter function for 13 weeks, however takes into account pathway, work assist/bonus, full outcomes

In [95]:
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

In [97]:
def get_26_outcome(jobSeekerEntry, num_codes, den_codes, com_codes): # takes a row (groupBy on Job_Seeker_ID, list of eligible numerator phase codes + denominator phase codes
    # sort jobSeekerEntry by COMMENCEMENT_DATE
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry # group containing the attributes for one job_seeker_ID
    id = group.index.values.astype(int)[0]
    num_26_full = 0
    den_26_full = 0
    num_26_path = 0 
    den_26_path = 0
    num_26_wrkast = 0 
    den_26_wrkast = 0
    df_26 = pd.DataFrame()
    # TEMP ----------- REMOVE
    df_26['Site_Name'] = [group.Site_Name.values[0]]
    df_26['Contract_ID'] = [group.Contract_ID.values[0]]
    df_26['specialist_site_type_code'] = [group.specialist_site_type_code.values[0]]

    # TEMP -------------------
    # place id into dataframe 
    df_26['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    # initialise the variables in df_26
    df_26['Num_26_full'] = [num_26_full]
    df_26['Den_26_full'] = 0
    df_26['Num_26_path'] = [num_26_path]
    df_26['Den_26_path'] = 0
    df_26['Num_26_wrkast'] = [num_26_wrkast]
    df_26['Den_26_wrkast'] = 0

    week_13 = pd.DateOffset(days = 91)
    week_26 = pd.DateOffset(days = 182) # 26 weeks -> because adding from commencement_date -> but maybe it should be from 13 week ?
    eduStr = ['DES 26wk Full Outcome Education']
    # Check number of entries in group
    targetStr = ['DES 26wk Full Bonus', 'DES 26wk Full Outcome', 'DES 26wk Pathway Outcome', 'DES 26wk Work Assist Outcome'] #valid outcomes
    num_entries = int(len(jobSeekerEntry))
    noClaim = 'na'

    valid_claims = ['Approved', 'Pending', 'Lodged'] 
    exitDate = group.EXIT_DATE.values[0] # if exit date exists ->  exit date is the same for all entries

    # now loop ->
    j = 0
    comDate =  group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_26 = comDate + week_26
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                # check suspension days is in the 2 year assessment period:
                 if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_26 = comDate + week_26 + sus_days
            else:
                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                
                # check if participant exits before claim_available_date ->
            
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_26):
                    if comDate > exitDate: # if commenced again after exit date:
                        # get the claim_rate_type -> 
                        if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                            num_26_full = 1
                            den_26_full = 1
                            df_26['Num_26_full'] = [num_26_full]
                            df_26['Den_26_full'] = 1
                            return df_26
                        elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                            num_26_wrkast = 1
                            den_26_wrkast = 1
                            df_26['Num_26_wrkast'] = [num_26_wrkast]
                            df_26['Den_26_wrkast'] = 1
                            return df_26
                        elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                            num_26_path = 1
                            den_26_path = 1
                            df_26['Num_26_path'] = [num_26_path]
                            df_26['Den_26_path'] = 1
                            return df_26

                    else:
                        num_26_full = 0
                        den_26_full = 0
                        num_26_path = 0 
                        den_26_path = 0
                        num_26_wrkast = 0 
                        den_26_wrkast = 0

                else: # check claim type 
                    if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                        num_26_full = 1
                        den_26_full = 1
                        df_26['Num_26_full'] = [num_26_full]
                        df_26['Den_26_full'] = 1
                        return df_26
                    elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                        num_26_wrkast = 1
                        den_26_wrkast = 1
                        df_26['Num_26_wrkast'] = [num_26_wrkast]
                        df_26['Den_26_wrkast'] = 1
                        return df_26
                    elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                        num_26_path = 1
                        den_26_path = 1
                        df_26['Num_26_path'] = [num_26_path]
                        df_26['Den_26_path'] = 1
                        return df_26
            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_26: # duration in assistance is less than 26 weeks
                    if comDate > exitDate: # if commenced again after exit date:
                        num_26_full += 0
                        den_26_full = 1


                    else: #just exited before period_26 
                        num_26_full = 0
                        den_26_full = 0
                        num_26_path = 0 
                        den_26_path = 0
                        num_26_wrkast = 0 
                        den_26_wrkast = 0
                   
                else: # check duration in assistance is at least period_26
                    num_26_full += 0
                    den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

            elif group.Claim_Rate_Type.values[j] == noClaim: 
                
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_26: # duration in assistance is less than 26 weeks
                    num_26_full = 0
                    den_26_full = 0
                    num_26_path = 0 
                    den_26_path = 0
                    num_26_wrkast = 0 
                    den_26_wrkast = 0
                else: # just denominate in the full outcome
            
                    num_26_full += 0 
                    den_26_full = 1
      
            else: # either 13 or 52 week claim
                num_26_full = 0
                

        else: # has not commenced ->
            
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_26 = comDate + week_26
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_26 = comDate + week_26 + sus_days
                
            # check if code is SUS_EA: (Counted in denominator if completed more than 26 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_26:
                    if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_26 <= startBounds) or (period_26 >= endBounds):
                                num_26_full = 0 
                                den_26_full = 0 # out of bounds
                        else:
                            # Check the claim type ->
                            if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                num_26_full = 1
                                den_26_full = 1
                                df_26['Num_26_full'] = [num_26_full]
                                df_26['Den_26_full'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                num_26_wrkast = 1
                                den_26_wrkast = 1
                                df_26['Num_26_wrkast'] = [num_26_wrkast]
                                df_26['Den_26_wrkast'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                num_26_path = 1
                                den_26_path = 1
                                df_26['Num_26_path'] = [num_26_path]
                                df_26['Den_26_path'] = 1
                                return df_26
                    else:
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_26 <= startBounds) or (period_26 >= endBounds):
                            num_26_full = 0 
                            den_26_full = 0 # out of bounds
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 26wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_26_full = 0 
                                den_26_path = 1
                            else: # check the claim types and denominate these
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # full outcome
                                    den_26_full = 1
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]:
                                    den_26_wrkast = 1
                                else: # NA claim
                                    den_26_full = 1

                else: # no claim_availble date or claim available date is less than period_26
                    if group.CLAIM_AVAILABLE_DATE.values[j] < period_26 and group.Claim_Rate_Type.values[j] == 'na':
                        # check duration in assistance
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_26: # exits after 26 weeks
                            num_26_full += 0
                            den_26_full = 1
                        else: # exits before 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                    else: # could be suspended but still record a claim ->
                        comDate = group.COMMENCEMENT_DATE.values[j]
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        period_26 = comDate + week_26
                        # check suspensions
                        if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                            # check suspension days is in the 2 year assessment period:
                            if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_26): # suspension in valid period ->
                                sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                                period_26 = comDate + week_26 + sus_days
                        else:
                            pass
 
                       
                        # claim_available_date could be valid and occur before period_26:
                        # claim rate types:
                        if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                            # check if participant exits before claim_available_date ->
                            
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if (exitDate < period_26):
                             

                                if comDate > exitDate:    # check if commences AFTER exit again:
                                    if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                        num_26_full = 1
                                        den_26_full = 1
                                        df_26['Num_26_full'] = [num_26_full]
                                        df_26['Den_26_full'] = 1
                                        return df_26
                                    elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                        num_26_wrkast = 1
                                        den_26_wrkast = 1
                                        df_26['Num_26_wrkast'] = [num_26_wrkast]
                                        df_26['Den_26_wrkast'] = 1
                                        return df_26
                                    elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                        num_26_path = 1
                                        den_26_path = 1
                                        df_26['Num_26_path'] = [num_26_path]
                                        df_26['Den_26_path'] = 1
                                        return df_26 



                                else: # just exits before period_26
                                    num_26_full = 0
                                    den_26_full = 0
                                    num_26_path = 0 
                                    den_26_path = 0
                                    num_26_wrkast = 0 
                                    den_26_wrkast = 0

                            else: # check claim type 
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                    num_26_full = 1
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                    num_26_wrkast = 1
                                    den_26_wrkast = 1
                                    df_26['Num_26_wrkast'] = [num_26_wrkast]
                                    df_26['Den_26_wrkast'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                    num_26_path = 1
                                    den_26_path = 1
                                    df_26['Num_26_path'] = [num_26_path]
                                    df_26['Den_26_path'] = 1
                                    return df_26
                        elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_26: # duration in assistance is less than 26 weeks
                                if comDate > exitDate: # commences again after exiting
                                    
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = den_26_full
                                else:
                                    num_26_full = 0
                                    
                                
                            else: # check duration in assistance is at least period_26
                                num_26_full = 0
                                den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

                        elif group.Claim_Rate_Type.values[j] == noClaim: 
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_26: # duration in assistance is less than 26 weeks
                                num_26_full = 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0
                            else: # just denominate in the full outcome
                                num_26_full += 0 
                                den_26_full += 1
                        else: # either 14 or 52 week claim
                            num_26_full = 0
                            
                
                                        
            else: # TRN or EXT
                ext_tfn = ['Exited', 'Transferred']
                if group.PLACEMENT_STATUS_CODE.values[j] in ext_tfn:
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[j])

                    if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                        # check if participant exits before claim_available_date ->
                        
                    
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if (exitDate < period_26):
                            if comDate > exitDate: # commenced after exit
                                if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                    num_26_full = 1
                                    den_26_full = 1
                                    df_26['Num_26_full'] = [num_26_full]
                                    df_26['Den_26_full'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                    num_26_wrkast = 1
                                    den_26_wrkast = 1
                                    df_26['Num_26_wrkast'] = [num_26_wrkast]
                                    df_26['Den_26_wrkast'] = 1
                                    return df_26
                                elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                    num_26_path = 1
                                    den_26_path = 1
                                    df_26['Num_26_path'] = [num_26_path]
                                    df_26['Den_26_path'] = 1
                                    return df_26
                                
                            
                            else:
                                num_26_full += 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0

                        else: # check claim type 
                            if group.Claim_Rate_Type.values[j] == targetStr[1]: # Full Outcome
                                num_26_full = 1
                                den_26_full = 1
                                df_26['Num_26_full'] = [num_26_full]
                                df_26['Den_26_full'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[0] or group.Claim_Rate_Type.values[j] == targetStr[3]: # Bonus/WorkAst
                                num_26_wrkast = 1
                                den_26_wrkast = 1
                                df_26['Num_26_wrkast'] = [num_26_wrkast]
                                df_26['Den_26_wrkast'] = 1
                                return df_26
                            elif group.Claim_Rate_Type.values[j] == targetStr[2]:
                                num_26_path = 1
                                den_26_path = 1
                                df_26['Num_26_path'] = [num_26_path]
                                df_26['Den_26_path'] = 1
                                return df_26
                    elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_26 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_26: # duration in assistance is less than 26 weeks
                            if comDate > exitDate: 
                                num_26_full += 0
                                den_26_full = 1
                            else:
                                num_26_full += 0
                                den_26_full = 0
                                num_26_path = 0 
                                den_26_path = 0
                                num_26_wrkast = 0 
                                den_26_wrkast = 0
                        
                        else: # check duration in assistance is at least period_26
                            num_26_full = 0
                            den_26_full = 1 #education outcomes are counted in the denominator of the full outcome

                    elif group.Claim_Rate_Type.values[j] == noClaim: 
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_26: # duration in assistance is less than 26 weeks
                            num_26_full = 0
                            den_26_full = 0
                            num_26_path = 0 
                            den_26_path = 0
                            num_26_wrkast = 0 
                            den_26_wrkast = 0
                        else: # just denominate in the full outcome
                            num_26_full += 0 
                            den_26_full = 1
                          
                    else: # either 14 or 52 week claim
                        num_26_full += 0
                
            
                
        j += 1

    df_26['Num_26_full'] = [num_26_full]
    df_26['Den_26_full'] = den_26_full
    df_26['Num_26_path'] = [num_26_path]        
    df_26['Den_26_path'] = den_26_full - num_26_full
    df_26['Num_26_wrkast'] = [num_26_wrkast]
    df_26['Den_26_wrkast'] = den_26_wrkast
    return df_26

df_entry = g.get_group(4019304019)

f = get_26_outcome(df_entry, num_codes, den_codes, com_codes) # pass results to a new dataframe and append each time
f





                    


,Site_Name,Contract_ID,specialist_site_type_code,Job_Seeker_ID,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,AimBig Employment AUBURN,0212898K,AALL,4019304019,0,1,0,1,0,0


# aggregation function for pre-quarter function:
Pass each jobseeker id to the pre-quarter-26 function, aggregates results into a dataframe df_26

In [98]:
# adds outcome to df_26
def agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes):
    df_26 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_26', 'Den_26', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome



    ids = df_pre["Job_Seeker_ID"].unique().astype("int64")
    for id in df_pre['Job_Seeker_ID'].unique():
        if math.isnan(id):
            pass
        else:
            df_entry = g.get_group(id)
            f = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
            df_26 = df_26.append(f)
            

    return df_26 # returns output from get_26_outcome 

### Pre-quarterisation 26 output file:
Outputs to github repository 

In [99]:
### Testing the aggregate function for 13 outcomes
df_26 = agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes)


warnings.filterwarnings('ignore')

print('Number of IDs for 26 week outcomes: ' + str(len(df_26)))

# output file: 

df_26.to_csv('C:/Users/AManalo/star_ratings_new/df_26_new.csv')


Number of IDs for 26 week outcomes: 4970


In [101]:
df_26.head(5)

,Job_Seeker_ID,Num_26,Den_26,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,9386301003,NaN,NaN,AimBig Employment WOY WOY,0212812K,AALL,0.0,0.0,0.0,0.0,0.0,0.0
0,8653691003,NaN,NaN,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,1.0,0.0,1.0,0.0,0.0
0,7870290004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,1.0,0.0,1.0,0.0,0.0
0,6498977509,NaN,NaN,AimBig Employment LEICHHARDT,0212845J,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,6638331004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,2836891309,NaN,NaN,AimBig Employment CORIO,0212862K,AALL,0.0,1.0,0.0,1.0,0.0,0.0
0,8771045004,NaN,NaN,AimBig Employment FRANKSTON,0212818F,MENH,0.0,1.0,0.0,1.0,0.0,0.0
0,15325790,NaN,NaN,AimBig Employment MACKAY,0212803K,MUSK,0.0,1.0,0.0,1.0,0.0,0.0
0,690092007,NaN,NaN,AimBig Employment Pty Ltd SUNBURY,0212817E,AALL,0.0,1.0,0.0,1.0,0.0,0.0


Group the results of the post-quarter-26 function by contract level identifiers.
- Sum across each of the outcomes to get an aggregated count

The pathway denominator is calculated by subtracting the numerator full outcome from the full denominator, added to the existing pathway count

In [102]:
df_26_grouped = df_26.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code', 'Job_Seeker_ID']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

df_26_grouped = df_26_grouped.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

df_26_grouped['Den_26_path'] += df_26_grouped['Den_26_full'] - df_26_grouped['Num_26_full'] 

In [103]:
df_26_temp = df_26_grouped

In [104]:
df_26_grouped = df_26_grouped.drop(columns=['Num_26_full', 'Den_26_full', 'Num_26_path', 'Num_26_wrkast', 'Den_26_wrkast'])

In [105]:
df_concat = pd.concat([df_26_grouped, df_26_temp])
df_concat

Den_26_path  \
Site_Name                          Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE       0212804K    MENH                               9.0   
                                   0212805A    MUSK                               2.0   
AimBig Employment AUBURN           0212841E    AALL                              23.0   
                                   0212898K    AALL                              16.0   
AimBig Employment BALLARAT CENTRAL 0212859D    MENH                               3.0   
...                                                                               ...   
AimBig Employment WYONG            0212881B    AALL                               8.0   
Aimbig Employment LAUNCESTON       0212865B    (blank)                            2.0   
                                   0212893F    AALL                               3.0   
Aimbig Employment RICHMOND         0212850F    AALL                               2.0   
Aimbig Employment WINDSOR          0212903C    (blank)                            0.0   

                                                                          Num_26_full  \
Site_Name                          Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE       0212804K    MENH                               NaN   
                                   0212805A    MUSK                               NaN   
AimBig Employment AUBURN           0212841E    AALL                               NaN   
                                   0212898K    AALL                               NaN   
AimBig Employment BALLARAT CENTRAL 0212859D    MENH                               NaN   
...                                                                               ...   
AimBig Employment WYONG            0212881B    AALL                               7.0   
Aimbig Employment LAUNCESTON       0212865B    (blank)                            6.0   
                                   0212893F    AALL                               3.0   
Aimbig Employment RICHMOND         0212850F    AALL                               0.0   
Aimbig Employment WINDSOR          0212903C    (blank)                            0.0   

                                                                          Den_26_full  \
Site_Name                          Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE       0212804K    MENH                               NaN   
                                   0212805A    MUSK                               NaN   
AimBig Employment AUBURN           0212841E    AALL                               NaN   
                                   0212898K    AALL                               NaN   
AimBig Employment BALLARAT CENTRAL 0212859D    MENH                               NaN   
...                                                                               ...   
AimBig Employment WYONG            0212881B    AALL                              15.0   
Aimbig Employment LAUNCESTON       0212865B    (blank)                            8.0   
                                   0212893F    AALL                               6.0   
Aimbig Employment RICHMOND         0212850F    AALL                               2.0   
Aimbig Employment WINDSOR          0212903C    (blank)                            0.0   

                                                                          Num_26_path  \
Site_Name                          Contract_ID specialist_site_type_code                
AimBig Employment AITKENVALE       0212804K    MENH                               NaN   
                                   0212805A    MUSK                               NaN   
AimBig Employment AUBURN           0212841E    AALL                               NaN   
                                   0212898K    AALL                               NaN   
AimBig Employment BALLARAT CENTRAL 0212859D    MENH               

## Post-Quarterisation 26 week outcome
    - Not included if time to achieve an outcome is < 26 weeks. AKA commenced towards the end of the 2Y assessment period.
    - Using the quarter dates from the 13 week pre-quarterisation function 
    - maxNum = 1, maxDen = 8
    - Needs to commence before denominating.
    - If df_26 (pre-quarter), num=0 -> then only check the denominator
    - If df_26 (pre-quarter), num=1 -> then need to check when it numerates
    - If quarter falls outside of the 2Y period, then not included
Included: 
-	26 week Pathway Outcomes
-	26 week Full Outcome
o	With lodged, pending or approved status at the site 
-	26 week Bonus Outcomes/Work Assist 

Denominator: 
- In order to denominate: duration in assistance it at least 26 weeks - excluding periods of suspension.
- EXCEPTION: if ANCHOR_DATE = CLAIM_AVAILABLE_DATE (for 13 week outcome).
    Method: 
    1. If the claims in the current JobSeeker group are ONLY 26/52 week outcomes -> check for 13 week outcome. 
    2. Get 13 week claim (Full outcome).
    3. Check if ANCHOR_DATE = CLAIM_AVAILABLE_DATE (commencement_date) -> then only add 13 weeks to the ANCHOR_DATE for 26 week time in duration. 
    4. If ANCHOR_DATE != CLAIM_AVAILABLE_DATE, then check duration in assistance is at least 26 weeks (excluding suspensions) - add 13 weeks to 13_week CLAIM_AVAILABLE_DATE. -> don't need to check suspensions for the 13 week claim, as the claim has already been lodged/pending/approved. 
    - Check suspensions accordingly. 

- doesn't denominate if exits before 26 week duration in assistance


In [106]:
targetStr = ['DES 26wk Full Bonus', 'DES 26wk Full Outcome', 'DES 26wk Pathway Outcome', 'DES 26wk Work Assist Outcome'] #valid outcomes

grpList = set(grp.Claim_Rate_Type.values)

string_26 = next(i for i in targetStr if i in grpList) # this returns the index of the instance of a target string from the list targetStr


Get the EXIT_DATE for the 26 wk claim -> to check whether job seeker exits before claim 

In [107]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]


endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')

startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

def post_quarter_26(df_26, df_pre, g, qtrs):
    '''Post quarterisation performance for 26 week full outcomes. Receives df_pre as an input, and outputs the post_quarterised denominators for each eligible participant.
    df_26 is used to check if a participant has numerated, df_pre is used to feed in the job_seeker_id to 'g' - the groupBy object and check whether the job seeker denominates - if no numerator in the df_26 exists.'''
    # maximum amount of times they numerate/denominate:
    maxDen = 8
    maxNum = 1

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Num_26', 'Den_26', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # Dataframe template for result
    
    claimExists = False
    
    i = 0
    while i < len(df_26):
       
        js_id = df_26.Job_Seeker_ID.values[i]
        siteName = df_26.Site_Name.values[i]
        contractID = df_26.Contract_ID.values[i]
        specialistCode = df_26.specialist_site_type_code.values[i]
        num_26_full = df_26.Num_26_full.values[i] 
        den_26_full = df_26.Den_26_full.values[i]
        Den_26_full = 0
    
        Num_26_full = num_26_full
        
        # pathway
        num_26_path = df_26.Num_26_path.values[i] 
        den_26_path = df_26.Den_26_path.values[i]
        Den_26_path = 0
    
        Num_26_path = num_26_path
        
        #work assist/bonus
        num_26_wrkast = df_26.Num_26_wrkast.values[i] 
        den_26_wrkast = df_26.Den_26_wrkast.values[i]
        Den_26_wrkast = 0
    
        Num_26_wrkast = num_26_wrkast
        

        x = 0
        group = g.get_group(js_id)
        n = len(group)
        week_26 = pd.DateOffset(days = 182)
        stopLoop = False
        
        while x < n or not stopLoop:
            comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0], format='%Y/%m/%d %H:%M') # commencement date should be the first index 
            claim = group.Claim_Rate_Type.values[x]
            availDate = group.CLAIM_AVAILABLE_DATE.values[x]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[x], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
            invalid_date = pd.to_datetime('2001-01-01', format='%Y-%m-%d')

          
            if den_26_full == maxDen or den_26_path == maxDen or den_26_wrkast == maxDen:
                pass # return -> denominated maxDen = 8 times, or maxNum/maxDen
            
            if (num_26_full == 0 and den_26_full == 0):
                if (num_26_full == 0 and den_26_full == 0):
                
                    Num_26_full = 0
                    Den_26_full = 0
                    stopLoop = True
                    break 
      
        
            elif comDate <= invalid_date:
                Num_26_full = 0
                Den_26_full += 0
                Num_26_path = 0
                Den_26_path += 0
                
            else: 
               
                # looping through the outcomes
                if (group.PLACEMENT_STATUS_CODE.values[x] == 'Exited') and (group.SUSPENSION_DAYS.values[x] > 0): # current entry is showing Exited -> check if exitDate is  
                    period_26 = comDate + week_26 
                    if (group.SUSPENSION_FROM_DATE.values[x] >= comDate) and (group.SUSPENSION_ACTUAL_END_DATE.values[x] <= period_26): 
                        # then suspension is applicable to this outcome and period
                        period_26 = comDate + week_26 + pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[x]))
                       
                        # check if participant has exited before the 26 week outcome was due: 
                        if exitDate < period_26: 
                            # then don't include at all -> wipe all numerator/denominator
                            Num_26_full = 0 
                            Den_26_full += 0
                            Num_26_path = 0
                            Den_26_path += 0
                            pass
                    
                    

                # loop through the quarters and compare with claim_available_date -> check which quarter it occurs in
                else:
                    
                    if (claim == 'DES 26wk Full Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 26 week outcome claim exists - validate the number for the denominator -> reset
                        Den_26_full = 1 
                        Num_26_full = 1
                        pass
                    
                    elif (claim == 'DES 26wk Pathway Outcome') and (group.Claim_Status.values[x] == 'Approved'): # if 26 week outcome claim exists - validate the number for the denominator -> reset
                        Den_26_path = 1 
                        Num_26_path = 1
                        pass
                        
                    elif claim == 'DES 52wk Full Outcome' or claim == 'DES 13wk Full Outcome':
                        Num_26_full += 0 
                        Den_26_full += 0 
                        pass
                    # 26 week outcome may already be recorded -> or may be in later entries
                    
                    else:
                        period_26 = comDate + week_26
                    # check if a 26 week full outcome exists in ANY of the group values
                        
                    z = 0
                    s = 0
                    e = 1
                    while z < len(qtrs) and not stopLoop:
                        start = pd.to_datetime(qtrs[z][s])
                        end = pd.to_datetime(qtrs[z][1])
                        
                        if z == len(qtrs) - 1:  # This is the last iteration
                            stopLoop = True
                        if (Den_26_full == maxDen or Num_26_full == maxNum) or (Den_26_path == maxDen or Num_26_path == maxNum) or (Den_26_wrkast == maxDen or Num_26_wrkast == maxNum):
                            stopLoop = True
                            break
                        else:
                            if 'DES 26wk Full Outcome' in group.Claim_Rate_Type.values:
                                # if the iteration -> loop is at the beginning of the current group -> get the commencement date of the FIRST row for the 26 week full outcome: 
                                outcomeInd = (group.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[outcomeInd]
                                # get EXIT date for this here
                        
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[outcomeInd], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
                                ## TO-DO - investigate whether a SUSPENSION period occurs here and if exitDate <= period_26 (comDate + suspension_period + 26 weeks)
                            elif ('DES 26wk Full Outcome' not in group.Claim_Rate_Type.values ) and ('DES 26wk Pathway Outcome' in group.Claim_Rate_Type.values):    # all values are 26 week PATHWAY outcomes
                                pathwayInd = (group.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                                comDate = group.COMMENCEMENT_DATE.values[pathwayInd]
                    
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[pathwayInd], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
                            # check here how many times the participant denominates ->
                            if (comDate >= start) and (comDate <= end) and (group.PLACEMENT_STATUS_CODE.values[x] == 'Commenced'): # check if the commencement date is in the current quarter: 
                                
                                period_26 = comDate + week_26 # denominates if and only if at least 26 weeks has passed: 26 weeks + comDate -> falls into the current quarter, if not pass
                                if claim == 'DES 26wk Pathway Outcome':
                                    Den_26_path += 1
                                    
                            
                                elif (exitDate >= start) and (exitDate <= end) and (period_26 >= start) and (period_26 <= end): #exit date is in the current quarter -> then denominates and then exists
                                
                                    Den_26_full += 1 
                                    stopLoop = True
                                    break
                                elif (availDate >= start) and (availDate <= end) and (period_26 >= start) and (period_26 <= end): # in the current quarter -> exit date not counted
                                    Den_26_full += 1
                                    stopLoop = True
                                    break
                                   
                                #add here if exit before period_26
                                elif (exitDate < period_26):
                                
                                    Den_26_full = 0
                                    stopLoop = True
                                    break
                            


                            else: # commencement date has already passed, OR commencement date is later
                                # need to stop at participant EXIT_DATE - get exit date of the 26 week full outcome or pathway outcome:
                                exitDate = pd.to_datetime(group.EXIT_DATE.values[0], format='%d/%m/%Y  %H:%M' ) # exit date should be recorded FOR ALL
                                period_26 =  comDate + week_26
                                
                                if (comDate > end): # commencement date is later ->
                                    pass
                                    
                                elif exitDate < period_26:
                     
                                    Den_26_full = 0
                                    stopLoop = True
                                    break 
                                
                                else: 
                               
                                    # check if exitDate is within the current quarter being traversed through
                                    if (exitDate >= start) and (exitDate <= end):
                                        if period_26 >= start and period_26 <= end: # duration at least 26 weeks OCCURS during this quarter and they exit in this quarter
                                            Den_26_full += 1
                                            stopLoop = True
                                            break
                                        elif exitDate < period_26:
                                            Den_26_full = 0
                                            stopLoop = True
                                            break
                                        else:
                                
                                            Den_26_full += 1
                                            stopLoop = True
                                            break
                                            
                                    elif (period_26 >= start and period_26 <= end) or (period_26 < start) and not (start <= exitDate <= end):
                                        
                                        Den_26_full += 1

                    
                                       
                                    else:
                                        Den_26_full += 0
                                       
                                
                        z += 1
                        
                   
                        
                    
                

                  

                x += 1
        

        if Den_26_full >= maxDen or Den_26_path >= maxDen:
            Den_26_path = 8  # set threshold
        data = pd.DataFrame({'Job_Seeker_ID' : [js_id], 'Num_26_full' : num_26_full, 'Den_26_full' : Den_26_full, 'Num_26_path' : num_26_path, 'Den_26_path' : Den_26_path, 'Num_26_wrkast' : num_26_wrkast, 'Den_26_wrkast' : Den_26_wrkast, 'Site_Name' : [siteName], 'Contract_ID' : [contractID], 'specialist_site_type_code' : [specialistCode]})
        df_post = df_post.append(data)

            

            
        
        i += 1
            





    return df_post # returns post-quarterisation dataframe 




**Output file for post quarterisation 26 week**
- Aggregate the pre-quarter outcomes into dataframe x
- pass dataframe x to post-quarter-26 function
- groupby contract level identifiers
- sum outcomes for groupby operation

In [108]:

x = agg_outcomes_26(df_pre, g, num_codes, den_codes, com_codes)

df_post_26 = post_quarter_26(x, df_pre, g, qtrs)
df_post_26.to_csv('C:/Users/AManalo/star_ratings_new/df_post_26.csv')

df_post_26_grouped = df_post_26.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code', 'Job_Seeker_ID']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

df_post_26_grouped = df_post_26_grouped.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code']).agg({
    'Num_26_full': 'sum',
    'Den_26_full': 'sum',
    'Num_26_path': 'sum',
    'Den_26_path': 'sum',
    'Num_26_wrkast': 'sum',
    'Den_26_wrkast': 'sum'
})

The pathway denominator is calculated by subtracting the numerator full outcome from the full denominator, and adding the existing pathway denominator

In [109]:
df_post_26_grouped['Den_26_path'] += (df_post_26_grouped['Den_26_full'] - df_post_26_grouped['Num_26_full'])

In [110]:
df_post_26_grouped
df_post_26_grouped.to_csv('C:/Users/AManalo/star_ratings_new/26_week_outcomes_sites.csv')

**Testing post quaterisation 26 week function**

In [111]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')


x = agg_outcomes_26(jobSeekers, g, num_codes, den_codes, com_codes)
df_26_test = post_quarter_26(x, df_pre, g, qtrs)
df_26_test.head(10)

,Job_Seeker_ID,Num_26,Den_26,Site_Name,Contract_ID,specialist_site_type_code,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,Den_26_wrkast
0,7661633709,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,1.0,0.0,0.0,0.0,0.0
0,4125376109,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,2416123019,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,1358028909,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,8081302119,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,8322186509,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,136713909,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,688359609,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,1.0,0.0,0.0,0.0,0.0
0,4433333019,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0
0,8308143109,NaN,NaN,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0,0.0,0.0


## 52 week outcomes

### Pre-quarterisation function (52 week outcome)

In [112]:
startDate = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
endDate = pd.to_datetime('2022-10-07', format='%Y-%m-%d')

startBounds = '2020-10-01'
endBounds = '2022-10-07'

In [113]:

def get_52_outcome(jobSeekerEntry, num_codes, den_codes, com_codes): # takes a row (groupBy on Job_Seeker_ID, list of eligible numerator phase codes + denominator phase codes
    # sort jobSeekerEntry by COMMENCEMENT_DATE
    jobSeekerEntry = jobSeekerEntry.sort_values(by=['COMMENCEMENT_DATE'])
    group = jobSeekerEntry # group containing the attributes for one job_seeker_ID
    id = group.index.values.astype(int)[0]
    # initialise numerator and denominator 
    num_52_full = 0
    den_52_full = 0
    num_52_path = 0 
    den_52_path = 0
    df_52 = pd.DataFrame()
    # TEMP ----------- REMOVE
    df_52['Site_Name'] = [group.Site_Name.values[0]]
    df_52['Contract_ID'] = [group.Contract_ID.values[0]]
    df_52['specialist_site_type_code'] = [group.specialist_site_type_code.values[0]]

    # TEMP -------------------
    # place id into dataframe 
    df_52['Job_Seeker_ID'] = [group.Job_Seeker_ID.values[0]]
    # initialise the variables in df_52
    df_52['Num_52_full'] = [num_52_full]
    df_52['Den_52_full'] = 0
    df_52['Num_52_path'] = [num_52_path]
    df_52['Den_52_path'] = 0

   
    week_52 = pd.DateOffset(days = 364) # 52 weeks -> 26 + 26 = 52 weeks -> days
    eduStr = ['DES 52wk Full Outcome Education']
    # Check number of entries in group
    targetStr = ['DES 52wk Full Outcome', 'DES 52wk Pathway Outcome'] #valid outcomes
    num_entries = int(len(df_entry))
    noClaim = 'na'
    valid_claims = ['Approved', 'Pending', 'Lodged'] 
    exitDate = group.EXIT_DATE.values[0] # if exit date exists ->  exit date is the same for all entries

# now loop ->
    j = 0
    comDate =  group.COMMENCEMENT_DATE.values[0]
    exitDate = group.EXIT_DATE.values[0] 
    while j < len(group.COMMENCEMENT_DATE.values):
        # commencement check ->
        if group.PLACEMENT_STATUS_CODE.values[j] == 'Commenced':
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_52 = comDate + week_52
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                # check suspension days is in the 2 year assessment period:
                 if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_52): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_52 = comDate + week_52 + sus_days
            else:

                pass

            # claim rate types:
            if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                # check if participant exits before claim_available_date ->
            
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if (exitDate < period_52):
                    if comDate > exitDate: # if commenced again after exit date:
                        # get the claim_rate_type -> 
                        if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                            num_52_full = 1
                            den_52_full = 1
                            df_52['Num_52_full'] = [num_52_full]
                            df_52['Den_52_full'] = 1
                            return df_52

                        elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                            num_52_path = 1
                            den_52_path = 1
                            df_52['Num_52_path'] = [num_52_path]
                            df_52['Den_52_path'] = 1
                            return df_52

                    else:
                        num_52_full = 0
                        den_52_full = 0
                        num_52_path = 0 
                        den_52_path = 0


                else: # check claim type 
                    if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                        num_52_full = 1
                        den_52_full = 1
                        df_52['Num_52_full'] = [num_52_full]
                        df_52['Den_52_full'] = 1
                        return df_52

                    elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                        num_52_path = 1
                        den_52_path = 1
                        df_52['Num_52_path'] = [num_52_path]
                        df_52['Den_52_path'] = 1
                        return df_52
            elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_52 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_52: # duration in assistance is less than 52 weeks
                    if comDate > exitDate: # if commenced again after exit date:
                        num_52_full += 0
                        den_52_full = 1


                    else:
                        num_52_full = 0
                        den_52_full = 0
                        num_52_path = 0 
                        den_52_path = 0

                   
                else: # check duration in assistance is at least period_52
                    num_52_full += 0
                    den_52_full = 1 #education outcomes are counted in the denominator of the full outcome

            elif group.Claim_Rate_Type.values[j] == noClaim: 
                exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                if exitDate < period_52: # duration in assistance is less than 52 weeks
                    num_52_full = 0
                    den_52_full = 0
                    num_52_path = 0 
                    den_52_path = 0

                else: # just denominate in the full outcome
                    num_52_full += 0 
                    den_52_full = 1
            else: # either 14 or 52 week claim
                num_52_full = 0
                den_52_full = 0

        else: # has not commenced ->
            comDate = group.COMMENCEMENT_DATE.values[j]
            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
            period_52 = comDate + week_52
            # check suspensions
            if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension 
                # check suspension days is in the 2 year assessment period:
                if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_52): # suspension in valid period ->
                    sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                    period_52 = comDate + week_52 + sus_days
                
            # check if code is SUS_EA: (Counted in denominator if completed more than 52 weeks with us. Includes suspension period)
            if group.PLACE_REF.values[j] == 'SUS_EA':
                if group.CLAIM_AVAILABLE_DATE.values[j] >= period_52:
                    if (group.Claim_Rate_Type.values[j] in targetStr) and (group.Claim_Status.values[j] in valid_claims):
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_52 <= startBounds) or (period_52 >= endBounds):
                                num_52_full = 0 
                                den_52_full = 0 # out of bounds
                        else:
                            # Check the claim type ->
                            if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                num_52_full = 1
                                den_52_full = 1
                                df_52['Num_52_full'] = [num_52_full]
                                df_52['Den_52_full'] = 1
                                return df_52

                            elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                num_52_path = 1
                                den_52_path = 1
                                df_52['Num_52_path'] = [num_52_path]
                                df_52['Den_52_path'] = 1
                                return df_52
                    else:
                        endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')
                        startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')
                        if (period_52 <= startBounds) or (period_52 >= endBounds):
                            num_52_full = 0 
                            den_52_full = 0 # out of bounds
                        else:
                            if (group.Claim_Rate_Type.values[j] == 'DES 52wk Pathway Outcome'): #this outcome IS NOt in the denominator
                                den_52_full += 0 
                                den_52_path = 1
                            else: # check the claim types and denominate these
                                if group.Claim_Rate_Type.values[j] == targetStr[0]: # full outcome
                                    den_52_full = 1

                                else: # NA claim
                                    den_52_full = 1

                else: # no claim_availble date or claim available date is less than period_52
                    if group.CLAIM_AVAILABLE_DATE.values[j] < period_52 and group.Claim_Rate_Type.values[j] == 'na':
                        # check duration in assistance
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate > period_52: # exits after 52 weeks
                            num_52_full += 0
                            den_52_full = 1
                        else: # exits before 52 weeks
                            num_52_full = 0
                            den_52_full = 0
                    else: # could be suspended but still record a claim ->
                        comDate = group.COMMENCEMENT_DATE.values[j]
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        period_52 = comDate + week_52
                        # check suspensions
                        if group.SUSPENSION_DAYS.values[j] > 0: # valid suspension
                            # check suspension days is in the 2 year assessment period:
                            if (group.SUSPENSION_FROM_DATE.values[j] >= group.COMMENCEMENT_DATE.values[j]) and (group.SUSPENSION_ACTUAL_END_DATE.values[j] <= period_52): # suspension in valid period ->
                                sus_days = pd.DateOffset(days = int(group.SUSPENSION_DAYS.values[j]))
                                period_52 = comDate + week_52 + sus_days
                        else:
                            pass
 
                       
                        # claim_available_date could be valid and occur before period_52:
                        # claim rate types:
                        if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                            # check if participant exits before claim_available_date ->
                        
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if (exitDate < period_52):
                                # check if commences AFTER exit again:

                                if comDate > exitDate:
                                    if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                        num_52_full = 1
                                        den_52_full = 1
                                        df_52['Num_52_full'] = [num_52_full]
                                        df_52['Den_52_full'] = 1
                                        return df_52

                                    elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                        num_52_path = 1
                                        den_52_path = 1
                                        df_52['Num_52_path'] = [num_52_path]
                                        df_52['Den_52_path'] = 1
                                        return df_52 



                                else:
                                    num_52_full = 0
                                    den_52_full = 0
                                    num_52_path = 0 
                                    den_52_path = 0


                            else: # check claim type 
                                if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                    num_52_full = 1
                                    den_52_full = 1
                                    df_52['Num_52_full'] = [num_52_full]
                                    df_52['Den_52_full'] = 1
                                    return df_52

                                elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                    num_52_path = 1
                                    den_52_path = 1
                                    df_52['Num_52_path'] = [num_52_path]
                                    df_52['Den_52_path'] = 1
                                    return df_52
                        elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_52 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_52: # duration in assistance is less than 52 weeks
                                if comDate > exitDate: # commences again after exiting
                                    num_52_full += 0
                                    den_52_full = 1
                                else:
                                    num_52_full = 0
                                    den_52_full = 0
                                    num_52_path = 0 
                                    den_52_path = 0

                                
                            else: # check duration in assistance is at least period_52
                                num_52_full += 0
                                den_52_full = 1 #education outcomes are counted in the denominator of the full outcome

                        elif group.Claim_Rate_Type.values[j] == noClaim: 
                            exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                            if exitDate < period_52: # duration in assistance is less than 52 weeks
                                num_52_full = 0
                                den_52_full = 0
                                num_52_path = 0 
                                den_52_path = 0

                            else: # just denominate in the full outcome
                                num_52_full += 0 
                                den_52_full = 1
                        else: # either 14 or 52 week claim
                            num_52_full = 0
                            den_52_full = 0
                
                                        
            else: # TRN or EXT
                ext_tfn = ['Exited', 'Transferred']
                if group.PLACEMENT_STATUS_CODE.values[j] in ext_tfn:
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[j])

                    if group.Claim_Rate_Type.values[j] in targetStr and group.Claim_Status.values[j] in valid_claims: # valid claim
                        # check if participant exits before claim_available_date ->
                    
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if (exitDate < period_52):
                            if comDate > exitDate: # commenced after exit
                                if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                    num_52_full = 1
                                    den_52_full = 1
                                    df_52['Num_52_full'] = [num_52_full]
                                    df_52['Den_52_full'] = 1
                                    return df_52

                                elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                    num_52_path = 1
                                    den_52_path = 1
                                    df_52['Num_52_path'] = [num_52_path]
                                    df_52['Den_52_path'] = 1
                                    return df_52
                                
                            
                            else:
                                num_52_full = 0
                                den_52_full = 0
                                num_52_path = 0 
                                den_52_path = 0


                        else: # check claim type 
                            if group.Claim_Rate_Type.values[j] == targetStr[0]: # Full Outcome
                                num_52_full = 1
                                den_52_full = 1
                                df_52['Num_52_full'] = [num_52_full]
                                df_52['Den_52_full'] = 1
                                return df_52

                            elif group.Claim_Rate_Type.values[j] == targetStr[1]:
                                num_52_path = 1
                                den_52_path = 1
                                df_52['Num_52_path'] = [num_52_path]
                                df_52['Den_52_path'] = 1
                                return df_52
                    elif group.Claim_Rate_Type.values[j] in eduStr and (group.CLAIM_AVAILABLE_DATE.values[j] >= period_52 or group.CLAIM_AVAILABLE_DATE.values[j] >= comDate):
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_52: # duration in assistance is less than 52 weeks
                            if comDate > exitDate: 
                                num_52_full += 0
                                den_52_full = 1
                            else:
                                num_52_full = 0
                                den_52_full = 0
                                num_52_path = 0 
                                den_52_path = 0


                        else: # check duration in assistance is at least period_52
                            num_52_full += 0
                            den_52_full = 1 #education outcomes are counted in the denominator of the full outcome

                    elif group.Claim_Rate_Type.values[j] == noClaim: 
                        exitDate = pd.to_datetime(group.EXIT_DATE.values[j])
                        if exitDate < period_52: # duration in assistance is less than 52 weeks
                            num_52_full = 0
                            den_52_full = 0
                            num_52_path = 0 
                            den_52_path = 0

                        else: # just denominate in the full outcome
                            num_52_full += 0 
                            den_52_full = 1
                    else: # either 14 or 52 week claim
                        num_52_full = 0
                        den_52_full = 0
                
        j += 1

    df_52['Num_52_full'] = [num_52_full]
    df_52['Den_52_full'] = den_52_full
    df_52['Num_52_path'] = [num_52_path]        
    df_52['Den_52_path'] = den_52_path

    return df_52

df_entry = g.get_group(451780)

f = get_52_outcome(df_entry, num_codes, den_codes, com_codes) # pass results to a new dataframe and append each time
f






,Site_Name,Contract_ID,specialist_site_type_code,Job_Seeker_ID,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,AimBig Employment CALOUNDRA,0212875D,MUSK,451780,0,0,1,1


# aggregation function for pre-quarter function:

In [114]:
# adds outcome to df_52
def agg_outcomes_52(df_pre, g, num_codes, den_codes, com_codes):
    df_52 = pd.DataFrame(columns= ['Job_Seeker_ID', 'Num_52', 'Den_52', 'Site_Name', 'Contract_ID', 'specialist_site_type_code']) # empty
    # takes grouped by job_seeker_id dataframe, passes each row to get_no#_outcome



    ids = df_pre["Job_Seeker_ID"].unique().astype("int64")
    for id in df_pre['Job_Seeker_ID'].unique():
        if math.isnan(id):
            pass
        else:
            df_entry = g.get_group(id)
            f = get_52_outcome(df_entry, num_codes, den_codes, com_codes)
            df_52 = df_52.append(f)
            

    return df_52 # returns output from get_52_outcome 

**Pre-quarterisation 52 output file:**

In [115]:
### Testing the aggregate function for 13 outcomes
df_52 = agg_outcomes_52(df_pre, g, num_codes, den_codes, com_codes)


warnings.filterwarnings('ignore')

print('Number of IDs for 52 week outcomes: ' + str(len(df_52)))


df_52.to_csv('C:/Users/AManalo/star_ratings_new/df_52_new.csv')

Number of IDs for 52 week outcomes: 4970


## Post-Quarterisation 52 week outcome

In [116]:
qtrs = [['2020-10-01', '2020-12-31'], ['2021-01-01', '2021-03-31'], ['2021-04-01', '2021-06-30'], ['2021-07-01', '2021-09-30'], ['2021-10-01', '2021-12-31'], ['2022-01-03', '2022-03-31'], ['2022-04-01', '2022-06-30'], ['2022-07-01', '2022-09-30']]

endBounds = pd.to_datetime('2022-10-07', format='%Y-%m-%d')

startBounds = pd.to_datetime('2020-10-01', format='%Y-%m-%d')

def post_quarter_52(df_52, df_pre, g, qtrs): 
    '''Takes the same arguments as post_quarter_13, however df_52 is added. '''
    
    maxDen = 8

    df_post = pd.DataFrame(columns=['Job_Seeker_ID', 'Site_Name', 'Contract_ID', 'specialist_site_type_code', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'])
    qtrs = pd.DataFrame(qtrs, columns=["start", "end"]).apply(pd.to_datetime)

    for index, row in df_52.iterrows():
        num_52_full = 0
        num_52_path = 0
      
        num_52_full = row['Num_52_full']
        num_52_path = row['Num_52_path']
     
        Num_52_full = row['Num_52_full']
        Num_52_path = row['Num_52_path']
      
        js_id = row['Job_Seeker_ID']
        den_52_full = row['Den_52_full']
        den_52_path = row['Den_52_path']
      
        Den_52_Full = row['Den_52_full']
        siteName = row['Site_Name']
        contractID = row['Contract_ID']
        specialistCode = row['specialist_site_type_code']
        
        group = g.get_group(js_id)
        week_52 = pd.DateOffset(days=364)

        if (den_52_full == maxDen) or (den_52_path == maxDen):
            break

        for _, group_row in group.iterrows():
            comDate = pd.to_datetime(group_row['COMMENCEMENT_DATE'])
            claim = group_row['Claim_Rate_Type']
            availDate = group_row['CLAIM_AVAILABLE_DATE']
            exitDate = pd.to_datetime(group_row['EXIT_DATE'])

            period_52 = comDate + week_52

            if (group_row['SUSPENSION_DAYS'] > 0) and (group_row['SUSPENSION_FROM_DATE'] >= comDate) and (group_row['SUSPENSION_ACTUAL_END_DATE'] <= period_52): # if suspension exits and is after commencement day
                sus_days = pd.DateOffset(days=int(group_row['SUSPENSION_DAYS']))
                period_52 = comDate + week_52 + sus_days

            if exitDate < period_52:
                num_52_full = 0
                den_52_full = 0
                num_52_path = 0
                den_52_path = 0
 
                break
                
            # loop through the quarters
            z = 0
            s = 0
            e = 1
            while z < len(qtrs):
                start = qtrs.loc[z, "start"]
                end = qtrs.loc[z, "end"]

                if den_52_full == maxDen or den_52_path == maxDen:
                    break

                if num_52_full == 1 or num_52_path == 1:
                    
                    if num_52_full == 1:
                        if (comDate >= start) and (comDate <= end): # check if the commencement date is in the current quarter:
                        
                            period_52 = comDate + week_52 # denominates if and only if at least 13 weeks has passed: 13 weeks + comDate -> falls into the current quarter, if not pass
                                
                    
                            if (exitDate >= start) and (exitDate <= end) and (period_52 >= start): #exit date is in the current quarter -> then denominates and then exists
                                den_52_full += 1 
                                
                                
                                break
                            elif (availDate >= start) and (availDate <= end): # in the current quarter -> exit date not counted
                                den_52_full += 1
                                break
                                
                            elif (exitDate <= period_52):
                                # check if comDate is also after exitDate ->
                                
                                if comDate > exitDate: #if commenced again after exiting
                                    
                                    den_52_full += 1
                                else:
                                    den_52_full = 0 
                                    break 
                        
         
                    elif num_52_path == 1:
                        if comDate >= start:
                            if availDate >= start and availDate <= end and availDate <= period_52:
                                den_52_path += 1
                            elif availDate <= end:
                                den_52_path += 1
                            elif start >= period_52:
                                den_52_path += 1
                            else:
                                den_52_path = 0
                   
                else:
                    
                    exitDate = pd.to_datetime(group.EXIT_DATE.values[0])
                    comDate = pd.to_datetime(group.COMMENCEMENT_DATE.values[0])
                    if den_52_full >= 1:
                        
                      
                        period_52 = comDate + week_52 # denominates if and only if at least 13 weeks has passed: 13 weeks + comDate -> falls into the current quarter, if not pass
                        
                
                        if (exitDate >= start) and (exitDate <= end) and (period_52 >= start): #exit date is in the current quarter -> then denominates and then exists
                            den_52_full += 1 

                            break
                       
                            
                        elif (exitDate <= period_52):
                            # check if comDate is also after exitDate ->
                            
                            if comDate > exitDate:
                                
                                den_52_full += 1
                            else:
                                
                                den_52_full = 0 
                                break 
                        else: 
                            den_52_full += 1
                    
                           

                    elif den_52_path == 1:
                        if comDate >= start:
                            if period_52 <= end:
                                den_52_path += 1
                            else:
                                den_52_path = 0
                        elif period_52 <= start:
                            if exitDate >= start and exitDate <= end:
                                den_52_path += 1
                                break
                            else:
                                den_52_path += 1
               
                    else:
                        pass
                if den_52_full > 8:
                    den_52_full = 8
                    break
                elif den_52_path > 8:
                    den_52_path = 8
                    break

             
                
                z += 1

           
            data = pd.DataFrame({'Job_Seeker_ID': [js_id], 'Site_Name': [siteName], 'Contract_ID': [contractID], 'specialist_site_type_code': [specialistCode], 'Num_52_full': Num_52_full, 'Num_52_path': Num_52_path, 'Den_52_full': den_52_full, 'Den_52_path': den_52_path})
            df_post = df_post.append(data)
            break
            
      
    return df_post

jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')


x = agg_outcomes_52(jobSeekers, g, num_codes, den_codes, com_codes)
df_52_test = post_quarter_52(x, df_pre, g, qtrs)
df_52_test


,Job_Seeker_ID,Site_Name,Contract_ID,specialist_site_type_code,Num_52_full,Den_52_full,Num_52_path,Den_52_path
0,4125376109,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0
0,2416123019,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,8.0,0.0,0.0
0,1358028909,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0
0,8081302119,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0
0,688359609,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,0.0,0.0,0.0
0,4433333019,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0
0,8308143109,AimBig Employment BROADMEADOW,0212815C,AALL,1.0,1.0,0.0,0.0
0,2920493109,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,0.0,0.0,0.0
0,7935743003,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,8.0,0.0,0.0
0,1650096209,AimBig Employment BROADMEADOW,0212815C,AALL,0.0,0.0,0.0,0.0


**Output file: post_quarter_52**
- Running the post_quarter_52 week function

In [117]:
df_post_52 = post_quarter_52(df_52, df_pre, g, qtrs)


df_post_52.to_csv('C:/Users/AManalo/star_ratings_new/df_post_52.csv') # WITH job_seeker_ID

In [118]:
df_post_52 = df_post_52.drop(columns=['Job_Seeker_ID']) # remove job_seeker_ID

# group by contract -> separate file to repository
df_post_52 = df_post_52.groupby(['Site_Name', 'Contract_ID', 'specialist_site_type_code'])['Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()
df_post_52['Den_52_path'] = (df_post_52['Den_52_full'] - df_post_52['Num_52_full']) + df_post_52['Num_52_path']
df_post_52.to_csv('C:/Users/AManalo/star_ratings_new/52_week_outcomes_sites.csv')


#### Caseload/Missing
- Caseload - identify low job seekers (<20 caseload) -> insufficient load -> identify contracts
- Count case load -> overall 
- missing data -> gaps we need to fill 
- Identify people that are not getting an outcome at all 
- Find proportion of data that needs to be cleaned (%) 



1) Caseload File


In [119]:
# total caseload:

#naVal = df_cases.isna().sum() * 100 / len(df_cases)
#missValDf = pd.DataFrame({'column_name': df_cases.columns, 'percent_missing': naVal})

#print('total cases: ' + str(len(df_cases)))
#missValDf.sort_values(by=['percent_missing'], ascending=False).head(17)



2) Claims File

In [120]:

# naVal = df_claims.isna().sum() * 100 / len(df_claims)
# missValDf = pd.DataFrame({'column_name': df_claims.columns, 'percent_missing': naVal})

# print('total claims: ' + str(len(df_claims)))
# missValDf.sort_values(by=['percent_missing'], ascending=False).head(3)



### Participants with no outcome recorded for Dec Star Rating 2021 - Job_Seeker_ID

In [121]:
# # participants without an outcome - get the ones with an outcome from the 'find' variable
# outcome = df_pre["Job_Seeker_ID"].drop_duplicates()
# df_pre_temp = df_pre_temp["Job_Seeker_ID"].drop_duplicates()
# vals = df_pre_temp.isin(outcome)
# # list of participants (Job_Seeker_ID) without an outcome

# # getting the false values:
# no_outcome = (vals != True).index.values



### Proportion of Data that needs Cleaning
1. Listing distinct values for each attribute and inspecting manually

In [122]:
df_pre.columns

Index(['Job_Seeker_ID', 'REFERRAL_PHASE_CODE', 'PLACEMENT_STATUS_CODE',
       'PLACEMENT_TYPE', 'Program', 'Contract_ID', 'Site_Name', 'ESA_Code',
       'specialist_site_type_code', 'COMMENCEMENT_DATE', 'EXIT_DATE',
       'Claim_ID', 'Claim_Rate_Type', 'Claim_Status', 'CLAIM_START_DATE',
       'CLAIM_AVAILABLE_DATE', 'CLAIM_END_DATE', 'SUSPENSION_FROM_DATE',
       'SUSPENSION_ACTUAL_END_DATE', 'OUTCOME_TYPE', 'SUSPENSION_DAYS',
       'PLACE_REF'],
      dtype='object')

In [123]:
# place_seq_num = df_pre['PLACEMENT_SEQUENCE_NUMBER'].drop_duplicates().values
# print('place_seq_num: ', np.sort(place_seq_num))

# job_seeker_id = df_pre['Job_Seeker_ID'].drop_duplicates().values
# #print('job_seekers: ', job_seeker_id)

# place_stat_cd = df_pre['PLACEMENT_STATUS_CODE'].drop_duplicates().values
# print('place_stat_cd: ', place_stat_cd)

# ref_phase_cd = df_pre['REFERRAL_PHASE_CODE'].drop_duplicates().values
# print('ref phase code: ', ref_phase_cd)

# commence_dt = df_pre['COMMENCEMENT_DATE'].drop_duplicates().values
# #print('commence_date: ', commence_dt)

# claim_id = df_pre['Claim_ID'].drop_duplicates().values
# print('claim_id: ', claim_id)

# claim_rate = df_pre['Claim_Rate_Type'].drop_duplicates().values
# print('claim_rate: ', claim_rate)

# claim_stat = df_pre['Claim_Status'].drop_duplicates().values
# print('claim stat: ', claim_stat)

# claim_start = df_pre['CLAIM_START_DATE'].drop_duplicates().values
# #print('claim_start: ', claim_start)

# claim_avail = df_pre['CLAIM_AVAILABLE_DATE'].drop_duplicates().values
# #print('claim_avail: ', claim_avail)

# claim_end = df_pre['CLAIM_END_DATE'].drop_duplicates().values
# #print('claim_end: ', claim_end)

# suspen_start = df_pre['SUSPENSION_FROM_DATE'].drop_duplicates().values
# #print('suspen_start_date: ', suspen_start)

# suspen_end = df_pre['SUSPENSION_ACTUAL_END_DATE'].drop_duplicates().values
# #print('suspen end: ', suspen_end)

# outcome_type = df_pre['OUTCOME_TYPE'].drop_duplicates().values
# print('outcome_type: ', outcome_type)



- Missing values for placement_sequence_number: 13, 15, 16, 17, 18, 19
- Min: 1, Max: 20

- Incorrect spelling for claim_rate: 'Post Placement Special Payment - DES DMS Job Seeke' - 'Job Seeker?'

# Testing

In [124]:
import io, os, sys, types
import unittest
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

### Coffs Harbour Test for 26 week outcomes
Contract: 0212826F

In [125]:
jobSeekers = pd.read_csv('C:/Users/AManalo/star_ratings_new/jobseeker_test.csv')
# job seekers
jobSeekers = jobSeekers["Job_Seeker_ID"].unique()

len(jobSeekers)

41

In [126]:
class TestNotebook(unittest.TestCase):
    # 13 week test cases:                                                                      
    def test_edu_13(self):
        '''If DES 13wk Full Outcome Education = Claim_Rate_Type, then den=1, num=0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome Education' in df_entry.Claim_Rate_Type.values:
                out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                num = out['Num_13'][0]
                den = out['Den_13'][0]

                assert (num == 0), "Education 13 outcome should NOT numerate."
                assert (den == 1), "Education 13 outcome MUST denominate."

    def test_valid_13(self):
        '''Claim status should be Approved, Lodged, Pending for it to numerate and claim_available_date AFTER or EQUAL TO commencement_date'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                if df_entry.Claim_Status.values[ind_13] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_13] >= df_entry.COMMENCEMENT_DATE.values[ind_13]):
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 1), "Valid 13 full outcome, must numerate."
                    assert (den == 1), "Valid 13 full outcome, must denominate."

                else: # Claim Status is not valid
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "Invalid 13 full outcome, doesn't numerate."
                    assert (den == 1), "Invalid 13 full outcome, must denominate."



    def test_13_date(self):
        '''If 13 week claim available date is BEFORE commencement date -> then 0/0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                if df_entry.CLAIM_AVAILABLE_DATE.values[ind_13] < df_entry.COMMENCEMENT_DATE.values[ind_13]:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "Invalid Claim_Available_Date, doesn't numerate."
                    assert (den == 0), "Invalid Claim_Available_Date, doesn't denominate."

    def test_13_exit(self):
        '''If a participant exits before 13 week claim is due, then Num=0,Den=0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_13 = (df_entry.Claim_Rate_Type.values == 'DES 13wk Full Outcome').argmax()
                claim_avail_date = pd.to_datetime(df_entry.CLAIM_AVAILABLE_DATE.values[ind_13])
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_13])
                if exit_date < claim_avail_date:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]
                    try: 
                        assert (num == 0), "Exits before claim, doesn't numerate."
                        assert (den == 0), "Exits before claim, doesn't denominate."
                    except AssertionError as e:
                        print(e)
                        print(df_entry.Job_Seeker_ID)
                    


    def test_13_path(self):
        '''If a 13 week pathway outcome is recorded, then Num=0, Den=1. Participant denominates.'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 13wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                pass
            else:
                if 'DES 13wk Pathway Outcome' in df_entry.Claim_Rate_Type.values:
                    out = get_13_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_13'][0]
                    den = out['Den_13'][0]

                    assert (num == 0), "13 wk pathway doesn't numerate."
                    assert (den == 1), "13 wk pathway must denominate."
    
    ###### 26 week outcome tests
    def test_26_edu(self):
        '''26 week Education outcomes are not included. Num=0, Den=1'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Full Outcome Education' in df_entry.Claim_Rate_Type.values:
                out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                num = out['Num_26_full'][0]
                den = out['Den_26_full'][0]
                assert (num == 0), "Education 26 outcome not valid. Doesn't numerate."
                assert (den == 1), "Education 26 outcome included in the denominator."

    def test_26_valid_path(self):
        '''26 week Pathway outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses.'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Pathway Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Pathway Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 1), "Valid 26 pathway, must numerate."
                    assert (den == 1), "Valid 26 pathway, must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid 26 pathway, doesn't numerate."
                    assert (den == 1), "Invalid 26 pathway, must denominate."

    def test_26_valid_full(self):
        '''26 week Full outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses.'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Full Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 1), "Valid 26 outcome, must numerate." 
                    assert (den == 1), "Valid 26 outcome, must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid claim status, doesn't numerate."
                    assert (den == 1), "Invalid claim status, must denominate."
            

    def test_26_valid_bonus(self):
        '''26 week Bonus or 26 week work assist outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses'''
                
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if 'DES 26wk Bonus' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Bonus').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26_wrkast'][0]
                    assert (num == 1), "Valid bonus claim, must numerate." 
                    assert (den == 1), "Valid bonus claim, must denominate."

                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]
                    assert (num == 0), "Invalid claim status, should not numerate."
                    assert (den == 1), "Invalid claim status, must denominate."
                
            elif 'DES 26wk Work Assist Outcome' in df_entry.Claim_Rate_Type.values:
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Work Assist Outcome').argmax()
                if df_entry.Claim_Status.values[ind_26] in ['Approved', 'Lodged', 'Pending'] and (df_entry.CLAIM_AVAILABLE_DATE.values[ind_26] >= df_entry.COMMENCEMENT_DATE.values[ind_26]):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26)_wrkast'][0]

                    assert (num == 1), "Valid 26wk Work Assist must numerate."
                    assert (den == 1), "Valid 26wk Work Assist must denominate."
                else: # Claim Status is not valid
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_wrkast'][0]
                    den = out['Den_26_wrkast'][0]

                    assert (num == 0), "Invalid 26 work assist claim status, no outcome."
                    assert (den == 1), "Invalid 26 work assist claim status, must denominate."

    def test_26_exit(self):
        '''If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0'''
        for x in jobSeekers:
            df_entry = g.get_group(x)
            if ('DES 26wk Full Outcome' in df_entry.Claim_Rate_Type.values):
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Full Outcome').argmax()
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_26])
                claim_avail_date = pd.to_datetime(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
                if (exit_date) < (claim_avail_date):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_full'][0]
                    den = out['Den_26_full'][0]

                    assert (num == 0), "Exits before 26 full claim, doesn't numerate."
                    assert (den == 0), "Exits before 26 full claim, doesn't denominate."
            
            elif ('DES 26wk Pathway Outcome' in df_entry.Claim_Rate_Type.values):
                ind_26 = (df_entry.Claim_Rate_Type.values == 'DES 26wk Pathway Outcome').argmax()
                exit_date = pd.to_datetime(df_entry.EXIT_DATE.values[ind_26])
                claim_avail_date = pd.to_date(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
                if (exit_date) < (claim_avail_date):
                    out = get_26_outcome(df_entry, num_codes, den_codes, com_codes)
                    num = out['Num_26_path'][0]
                    den = out['Den_26_path'][0]
          
                    assert (num == 0), "Exits before 26 pathway claim, doesn't numerate."
                    assert (den == 0), "Exits before 26 pathway claim, doesn't denominate."

if __name__ == '__main__':
    main = TestNotebook()
    # execute unit test
    import sys

    suite = unittest.TestLoader().loadTestsFromTestCase(TestNotebook)
    unittest.TextTestRunner(verbosity=4, stream=sys.stderr).run(suite)

test_13_date (__main__.TestNotebook)
If 13 week claim available date is BEFORE commencement date -> then 0/0 ... FAIL
test_13_exit (__main__.TestNotebook)
If a participant exits before 13 week claim is due, then Num=0,Den=0 ... ok
test_13_path (__main__.TestNotebook)
If a 13 week pathway outcome is recorded, then Num=0, Den=1. Participant denominates. ... ok
test_26_edu (__main__.TestNotebook)
26 week Education outcomes are not included. Num=0, Den=1 ... ok
test_26_exit (__main__.TestNotebook)
If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0 ... ERROR
test_26_valid_bonus (__main__.TestNotebook)
26 week Bonus or 26 week work assist outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses ... ok
test_26_valid_full (__main__.TestNotebook)
26 week Full outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses. ... 

Exits before claim, doesn't numerate.
39284    1650096209
39283    1650096209
38217    1650096209
39253    1650096209
39252    1650096209
            ...    
38519    1650096209
39024    1650096209
37587    1650096209
37648    1650096209
39495    1650096209
Name: Job_Seeker_ID, Length: 72, dtype: int64


ok
test_26_valid_path (__main__.TestNotebook)
26 week Pathway outcomes are included. Num=1, Den=1. Also tests the valid outcome statuses. ... FAIL
test_edu_13 (__main__.TestNotebook)
If DES 13wk Full Outcome Education = Claim_Rate_Type, then den=1, num=0 ... ok
test_valid_13 (__main__.TestNotebook)
Claim status should be Approved, Lodged, Pending for it to numerate and claim_available_date AFTER or EQUAL TO commencement_date ... FAIL

ERROR: test_26_exit (__main__.TestNotebook)
If a participant exits before 26 week claim is due, then num_26 = 0, den_26 = 0
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\AManalo\AppData\Local\Temp\ipykernel_5080\2425717738.py", line 199, in test_26_exit
    claim_avail_date = pd.to_date(df_entry.CLAIM_AVAILABLE_DATE.values[ind_26])
  File "C:\Users\AManalo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\p